**Notes**:  
The purpose of the file is to assemble final sheet varians from separate tables.

In [218]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Functions

In [219]:
# Fuction to show missed data in a table
def missing_values_tab(df):
    #Count missed values
    mis_val = df.isnull().sum()
    #Count share of missed values of all data 
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mv_table = mv_table.rename(columns = {0 : 'missed_values', 1 : '%_of_all'})
    
    mv_table['data_type'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='missed_values',ascending=False)
    print ("Dataframe contains " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " strings.\n")
    print("It has  " + str(mv_table.shape[0]) +" columns with missed values")    
    return display(mv_table) 

In [220]:
def column_name_formatter(col):
    """used for delete prefixes 'Data_m_' in columns' names
    """ 
    if 'Data' in col:
        parts = col.split('_')
        new_name = parts[-1]
        return new_name
    else:
        return col

In [221]:
# Функция для очистки строк от префикса '!bp_'
def clear_rows(row):
        
    if re.match('^\!bp_\w+', str(row)):
        tmp = row.split('_')
        new_value = tmp[1]        
        return new_value
    else:
        return row

In [222]:
#Function to extract json into dataframes
def create_df(filelist):
    recipes_df = pd.DataFrame()
    dfs_list = []
    for file in filelist:
        with open(file) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            dic = (fj.flatten(data))
            df = pd.DataFrame.from_dict(dic, orient='index')
            dfs_list.append(df)
    for df in dfs_list:
        recipes_df = recipes_df.join(df, how = 'outer',lsuffix = 'x_',rsuffix = 'y_')
    #assign new column names
    column_numbers = list(range(len(recipes_df.columns)))
    recipes_df.columns = column_numbers
    #fill Nans
    #recipes_df = recipes_df.fillna(0)
    #remove prefix '!bp_'
    for col in recipes_df.columns:
        recipes_df[col] = recipes_df[col].apply(clear_rows)  
    #transpose dataframe
    recipes_df = recipes_df.T
    #-------------------------------------
    cols_to_drop = []
    for col in recipes_df.columns:
        if 'VisualParameters' in col or 'FXSettings' in col: #drop columns with visual parameters
            cols_to_drop.append(col)
        elif 'Data_Comment' in col or 'PrototypeLink' in col:  #drop columns with dev comments
            cols_to_drop.append(col)
        elif 'Meta_ShadowDeleted' in col: #drop trash column
            cols_to_drop.append(col)
        elif 'JsonPath' in col or 'PropertyPath' in col or 'OwnerString' in col: #drop columns with fileways
            cols_to_drop.append(col)
        elif 'UI' in col or 'Sound' in col or 'Icon' in col: #drop columns with sound, ui properties
            cols_to_drop.append(col)
    recipes_df = recipes_df.drop(cols_to_drop, axis = 1)
    #add column Filepath
    recipes_df['Filepath'] = filelist
    return recipes_df

In [223]:
# Function to get rid of characters between curly brackets
""" Sometimes not working correctly
def format_description(row):
    if '{' in row:
        row = re.sub(r'\s*{.*}\s*','',row)
    return row
"""
def format_description(row):   
    openining_braces = '\{'
    closing_braces = '\}'
    text_between = '.*?'
    x = re.sub(f'[{openining_braces}]{text_between}[{closing_braces}]','',row)
    #get rid of html tags
    x = re.sub('<[^>]*>',"",x)
    return x

In [224]:
#Function to find columns with string keys
def match_pattern(row):
        if re.match('^\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
   
        
match_pattern('ca24785c24384d142a6c22d7585bb3a6, BlueprintWeaponEnchantment')
"""
#finding columns with string keys
stringkey_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
print(stringkey_cols)   
    """

'\n#finding columns with string keys\nstringkey_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(match_pattern).any()\n        if a:            \n            stringkey_cols.append(col)\nprint(stringkey_cols)   \n    '

In [225]:
#Function to find columns with asset ids
def find_assetIds_cols(row):
        if re.match('^[a-zA-Z0-9]{32}$', str(row)):
            return True   
        else:
            return False      

"""
#finding columns with asset ids
assetIds_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()
        if a:            
            assetIds_cols.append(col)
print(assetIds_cols)   
    """

'\n#finding columns with asset ids\nassetIds_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()\n        if a:            \n            assetIds_cols.append(col)\nprint(assetIds_cols)   \n    '

In [226]:
#Function to find columns with context actions, containing $ signs
def find_context_columns(row):
        if re.match('^\$[A-Za-z0-9]+\$\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
"""
#find columns with rows containing $$
context_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_context_columns).any()
        if a:            
            context_cols.append(col)
print(context_cols)   
    """

'\n#find columns with rows containing $$\ncontext_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_context_columns).any()\n        if a:            \n            context_cols.append(col)\nprint(context_cols)   \n    '

In [227]:
def find_item_type(row):
    """Function to return item type from the column "Data_$type"
    """
    tmp = str(row).split(',')
    data_type = tmp[1]    
    parts = re.split('(?<=[a-z])(?=[A-Z])', data_type)
    item_type = parts[-1]   
    return item_type

In [228]:
def create_text_columns(df, index_df, columns_in, column_out, format_text=False):
    """ Function to create text columns with names and descriptions got from 
        stringkeys in index file in the form of dict
        """
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column
    column_out = str(column_out )
    index_df = index_df # index_dict object got from parsing json file with text information
    index_dict = dict(zip(index_df['index'],index_df['text'])) #dict for mapping text information between two tables    
    
    #_____________________________Get items names and descriptions__________________________ 
    
    for col in cols_names:
        list_of_ids = df[col].unique() # list of stringkeys
        #filter out empty elements
        list_of_ids = list(filter(None,list_of_ids))
        for id in list_of_ids:
            try: #if stringkey is in index file return text
                df.loc[df[col]==id, column_out] = df[col].map(index_dict)
            except: #if not return empty row
                df.loc[df[col]==id, column_out] = ''          
    
         # fill missed values with empty row       
    df[column_out] = df[column_out].fillna('')
    # format description to get rid of {}
    if format_text == True:
        df[column_out] = df[column_out].apply(format_description)

   
    return df

In [229]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [230]:
def find_properties(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False
def select_columns(df, keywords):
    """function to select columns to keep from a dataframe and return 4 lists of
        stringkey columns, assetids columns, misc columns and combined list of columns
    Arguments: df - dataframe to choose columns from, 
                keywords - list of keywords or names of columns to select for misc list    
    """
    selected_cols = [] # list of needed columns
    assetIds_cols = [] #list of columns with asset ids
    stringkey_cols = [] # list of columns with string keyes
    misc_cols = [] # list of misc columns
    property_cols = [] # list of property columns
    intro_cols = []   # first columns: AssetId and Data_$type in any table

    def find_ids(col):
        if re.match('^[a-z0-9]{32}(?:$|[, A-Za-z])', str(col)):
            return True
        else:
            return False      

    #find columns with asset ids
    for col in df.columns:
        if col !='AssetId':
            a = df[col].apply(find_assetIds_cols).any()
            if a:            
                assetIds_cols.append(col)
        else:
            intro_cols.append(col)

    #find columns with string keys
    for col in df.columns:    
        a = df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
    
    #find columns with rows containing property values

    for col in df.columns: 
        # if name of the column is according to the pattern   
        if  re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(col)):
            a = df[col].apply(find_ids).any()
            if a:                        
        #and its values are assetIds keys  add to property list                 
                property_cols.append(col) 
      
    # find miscellanious columns
    if len(keywords)>0:
        keywords = keywords
        for keyword in keywords:
            for col in df.columns:
                if keyword in col:
                    misc_cols.append(col)
    #else:
        #misc_cols = [d for d in df.columns if d not in assetIds_cols if d not in stringkey_cols if d not in property_cols]
    #update assetIds_cols
    assetIds_cols = [d for d in assetIds_cols if d not in property_cols]
    #Must- have columns, 'AssetId' and 'Data_$type'
    intro_cols.append(df.columns[1]) #add Data_$type
    #create combined list            
    selected_cols = intro_cols + assetIds_cols+ property_cols + stringkey_cols + misc_cols + ['Filepath']
      
    return selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols

In [231]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    
       

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [232]:
def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
    item_cols = []
    for col in df.columns:
        try:
            if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                item_cols.append(col)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
    return item_cols

## Work Part

### Folders

In [233]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'

### Load dataframes

In [234]:
#loading support dataframes
assets_df = pd.read_csv(load_folder + r'\assets_df.csv')
index_df = pd.read_csv(load_folder + r'\index_df.csv')
index_df_ru = pd.read_csv(load_folder + r'\index_df_ru.csv')
#load loot table en version
loot_table = pd.read_csv(load_folder + r'\loot_table.csv')
#load loot table ru version
loot_table_ru = pd.read_csv(load_folder + r'\loot_table_ru.csv')
#load vendor table
vendor_table = pd.read_csv(load_folder + r'\vendor_items.csv')
#load item areas table
item_areas_df = pd.read_csv(load_folder + r'\item_areas_df.csv')


In [235]:
# english version equipment table
equip_items_df = pd.read_csv(load_folder + r'\equip_items_df.csv')
#english version weapon table
weapons_table = pd.read_csv(load_folder + r'\weapons.csv')
#english version armor table
armors_df = pd.read_csv(load_folder + r'\armors_df.csv')
#english version project items
project_items_en = pd.read_csv(load_folder + r'\project_items_df_eng.csv')
#english version dialog items
dialog_items_table_eng = pd.read_csv(load_folder + r'\dialog_items_table_eng.csv')
#english version, enemies_stats_eng
enemies_stats_eng = pd.read_csv(load_folder + r'\enemyships_stats_eng.csv')

#russian version equipment
equip_items_df_ru = pd.read_csv(load_folder + r'\equip_items_df_ru.csv')
#russian version equipment
weapons_table_ru = pd.read_csv(load_folder + r'\weapons_ru.csv')
#russian version armor table
armors_df_ru = pd.read_csv(load_folder + r'\armors_df_ru.csv')
#russian version project items
project_items_ru = pd.read_csv(load_folder + r'\project_items_df_ru.csv')
#russian versiom of dialog items
dialog_items_table_ru = pd.read_csv(load_folder + r'\dialog_items_table_ru.csv')
#russian version, enemies_stats_eng
enemies_stats_ru = pd.read_csv(load_folder + r'\enemyships_stats_ru.csv')

#load character class ability types table
class_ability_types = pd.read_csv(load_folder + r'\class_ability_types.csv')
homeworld_talents = pd.read_csv(load_folder + r'\homeworld_talents.csv')

### Clear null values

In [236]:
dfs = [loot_table, vendor_table,equip_items_df, equip_items_df_ru, weapons_table, weapons_table_ru, armors_df,
        armors_df_ru, loot_table_ru, project_items_ru, project_items_en, dialog_items_table_eng,
          dialog_items_table_ru ,item_areas_df, enemies_stats_eng, enemies_stats_ru, class_ability_types,
          homeworld_talents]
for df in dfs:
    df.fillna('',inplace=True)

### Merge tables

Merge ```equip_items_df``` with vendor_table for english version:

In [237]:
equip_items_df = equip_items_df.merge(vendor_table[['Item_Name','Vendor_Name']], 
                                      how = 'left', 
                                      on = ['Item_Name'], 
                                      suffixes=(None,'_y')).fillna('')
equip_items_df.loc[equip_items_df['Vendor_Name']!="",'Is_Vendor'] = True
equip_items_df.loc[equip_items_df['Vendor_Name']=="",'Is_Vendor'] = False

In [238]:
display(equip_items_df.head())

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Vendor_Name,Is_Vendor
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter2,True
1,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True
2,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True
3,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True
4,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,KasballicaMissionChapter2,True


Merge ```equip_items_df_ru``` with vendor_table for russian version:

In [239]:
vendor_table.head()

,AssetId,Item_Id,ProfitFactorCost,RepToUnlock,Count,Filepath,Vendor_Name,Item_Id_stringkey,Item_Name,Item_Name_Ru,Vendor_Name_Ru
0,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,20,9560,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект,Друзианцы_Глава_2
1,c330647e71a547309301a16b1904830b,56e3576b84384495b9015d5ec88217ac,35,15810,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,0e987eda-d900-4c72-bd08-59d86c67643c,Medikit,Медкомплект,Друзианцы_Глава_2
2,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,26,25230,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня,Друзианцы_Глава_2
3,c330647e71a547309301a16b1904830b,20650117b63acf740998b602e108c51a,32,28310,7,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,f54981ae-fa49-45b5-a97e-5c7695f74449,Krak Grenade,Крак-граната,Друзианцы_Глава_2
4,c330647e71a547309301a16b1904830b,b6daa597bb6b5114bb9e21ea1e855792,29,37730,5,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\VendorTables\DrusianChapter2_VendorTable_Advanced.jbp,DrusianChapter2,5f33b743-e2b7-4353-9f5b-29f6e4cd71ba,Onslaught,Резня,Друзианцы_Глава_2


In [240]:
equip_items_df_ru = equip_items_df_ru.merge(vendor_table[['Item_Name', 'Item_Name_Ru','Vendor_Name_Ru']], 
                                      how = 'left',
                                      left_on = ['Item_Name'],
                                      right_on = ['Item_Name_Ru'], 
                                      suffixes=(None,'_y')).drop(columns = ['Item_Name_y'], axis = 1).fillna('')
equip_items_df_ru.loc[equip_items_df_ru['Vendor_Name_Ru']!="",'Is_Vendor'] = True
equip_items_df_ru.loc[equip_items_df_ru['Vendor_Name_Ru']=="",'Is_Vendor'] = False

In [241]:
display(equip_items_df_ru.head())

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_2,True
1,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_4,True
2,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_4,True
3,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_4,True
4,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,МиссияКасбаллики_Глава_2,True


Merge ```armors_df``` with ```vendor_table```

In [242]:
#english version
armors_df = armors_df.merge(vendor_table[['Item_Name','Vendor_Name']], 
                                      how = 'left', 
                                      on = ['Item_Name'], 
                                      suffixes=(None,'_y')).fillna('')
armors_df.loc[armors_df['Vendor_Name']!="",'Is_Vendor'] = True
armors_df.loc[armors_df['Vendor_Name']=="",'Is_Vendor'] = False

In [243]:
#russian version
armors_df_ru = armors_df_ru.merge(vendor_table[['Item_Name', 'Item_Name_Ru','Vendor_Name_Ru']], 
                                      how = 'left',
                                      left_on = ['Item_Name'],
                                      right_on = ['Item_Name_Ru'], 
                                      suffixes=(None,'_y')).drop(columns = ['Item_Name_y'], axis = 1).fillna('')
armors_df_ru.loc[armors_df_ru['Vendor_Name_Ru']!="",'Is_Vendor'] = True
armors_df_ru.loc[armors_df_ru['Vendor_Name_Ru']=="",'Is_Vendor'] = False

In [244]:
display(armors_df.head())
display(armors_df_ru.head())

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Vendor_Name,Is_Vendor
0,9c163228266c4cc892dfb98086d27b52,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,35f5651a088d4824951002d4910922fd,,98b410bd49bd4b74b00c78b5b4fba32f,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",3177d0cf74384129877de82675c5355e,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,,,,,,,,,,,,,,,,,,,,70745aa6-22f8-4d76-bb74-868c49fe6b94,,fe562964-fdd5-4da2-9a2b-6071128e8647,,,Armours,Pattern,65.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldarGuardianMeshArmour_Item.jbp,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,,Medium,,0.0,,,None,Human,,,False
1,def2d2071f11402e92ac2de764c916d4,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,0072569196d64b4fa323b2dfb10f8141,,0fa0782686d949a389eefe83f63e4658,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",41ec0e9159a449afa2d12003538ee331,,,,,,,,,,,,,,,,,,,,,,,,670eeada-6adb-4b14-8961-b9aff601fde9,,3ae37fbb-2baa-4127-85e7-7e31e042dd39,,,Xeno,Pattern,30.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldariRuneArmour_Item.jbp,Rune Armour,Base armour property: +10% dodge if the wearer's Willpower is more than 50.,,,Light,,0.0,,,None,Human,,,False
2,acb1f898c944476e939767d80550f6f0,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,817c0cf8afde4c43afe70ff049221475,,0fa0782686d949a389eefe83f63e4658,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,590cb5c394ee4b5eb213121c96063dec,c7a53b43ac5f4f11a1aef612dd3f4f71,,,,,,,,,,,,,,,,,,,,,,,,,,,,,8b493338-dc15-4a4d-9ae1-056e206c2173,,,Armours,Common,25.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\ArmoredBodyglove2_Item.jbp,Improved Armoured Bodyglove,,,,Light,,0.0,,,None,Human,,,False
3,b7e91f16ac8e4fab993dc577dd9c6a09,"579ddce9e6b6d8e44b05e0715cc6674

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor
0,9c163228266c4cc892dfb98086d27b52,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,35f5651a088d4824951002d4910922fd,,98b410bd49bd4b74b00c78b5b4fba32f,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",3177d0cf74384129877de82675c5355e,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,,,,,,,,,,,,,,,,,,,,70745aa6-22f8-4d76-bb74-868c49fe6b94,,fe562964-fdd5-4da2-9a2b-6071128e8647,,,Armours,Pattern,65.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldarGuardianMeshArmour_Item.jbp,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Базовое свойство: +15% к броне, если значение Ловкости больше 80.\nСвойство мира: +15 к Ловкости.",,,Medium,,0.0,,,None,Human,,,,False
1,def2d2071f11402e92ac2de764c916d4,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,0072569196d64b4fa323b2dfb10f8141,,0fa0782686d949a389eefe83f63e4658,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",41ec0e9159a449afa2d12003538ee331,,,,,,,,,,,,,,,,,,,,,,,,670eeada-6adb-4b14-8961-b9aff601fde9,,3ae37fbb-2baa-4127-85e7-7e31e042dd39,,,Xeno,Pattern,30.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldariRuneArmour_Item.jbp,Рунический доспех,"Базовое свойство: +10% к уклонению, если значение Воли больше 50.",,,Light,,0.0,,,None,Human,,,,False
2,acb1f898c944476e939767d80550f6f0,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,817c0cf8afde4c43afe70ff049221475,,0fa0782686d949a389eefe83f63e4658,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,590cb5c394ee4b5eb213121c96063dec,c7a53b43ac5f4f11a1aef612dd3f4f71,,,,,,,,,,,,,,,,,,,,,,,,,,,,,8b493338-dc15-4a4d-9ae1-056e206c2173,,,Armours,Common,25.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\ArmoredBodyglove2_Item.jbp,Улучшенный бронированный гермокостюм,,,,Light,,0.0,,,None,Human,,,,False
3,b7e91f16ac8e4fab993dc577dd9c6a09,

Merge ```weapons_table``` with ```vendor_table```:

In [245]:
#English version
weapons_table = weapons_table.merge(vendor_table[['Item_Name','Vendor_Name']], 
                                      how = 'left', 
                                      on = ['Item_Name'], 
                                      suffixes=(None,'_y')).fillna('')
weapons_table.loc[weapons_table['Vendor_Name']!="",'Is_Vendor'] = True
weapons_table.loc[weapons_table['Vendor_Name']=="",'Is_Vendor'] = False

In [246]:
#russian version
weapons_table_ru = (weapons_table_ru.merge(vendor_table[['Item_Name_Ru','Vendor_Name_Ru']], 
                                      how = 'left',
                                      left_on = ['Название'],
                                      right_on = ['Item_Name_Ru'], 
                                      suffixes=(None,'_y')).drop(columns=['Item_Name_Ru'])
                                      .rename(columns = {'Vendor_Name_Ru':'Фракция_Продавец'})
                                      .fillna('')
)
weapons_table_ru.loc[weapons_table_ru['Фракция_Продавец']!="",'Фракционный_Предмет'] = 'Да'
weapons_table_ru.loc[weapons_table_ru['Фракция_Продавец']=="",'Фракционный_Предмет'] = 'Нет'
display(weapons_table_ru.head(2))

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет
1,cd5fa172af484872820ffbb0d713985b,[Модель Мезоа] Электродуговое ружье,При попадании в цель из электродугового ружья вы также атакуете всех противников в радиусе 2 клеток от цели.,,Сплошное,Нет,Двуручное,RangedWeaponry,Pattern,Электрический,0.0,1.0,13.0,20.0,4.0,18.0,Ranged,30.0,10.0,10.0,One,Одиночный выстрел,Одиночный выстрел из используемого дистанционного оружия.,2.0,SingleShot,Перезарядка,Перезаряжает выбранное оружие.,3.0,Reload,,,0.0,,,,0.0,,,,0.0,,Эксплораторы_Глава_2,Да


Add item location for english version:

In [247]:
loot_table.head()

,index,AssetId,Data_$type,Data_CargoVolumeAmount,Data_ContainerName,Data_Identify,Data_Items_0_Count,Data_Items_0_m_Item,Data_Items_1_Count,Data_Items_1_m_Item,Data_Items_2_Count,Data_Items_2_m_Item,Data_Items_3_Count,Data_Items_3_m_Item,Data_Items_4_Count,Data_Items_4_m_Item,Data_Items_5_Count,Data_Items_5_m_Item,Data_Items_6_Count,Data_Items_6_m_Item,Data_Items_7_Count,Data_Items_7_m_Item,Data_Items_8_Count,Data_Items_8_m_Item,Data_Items_9_Count,Data_Items_9_m_Item,Data_Setting,Data_Type,Data_m_Area,Filepath,Area_Name,Area_Name_Ru,Data_Items_0_m_Item_stringkey,Data_Items_1_m_Item_stringkey,Data_Items_2_m_Item_stringkey,Data_Items_3_m_Item_stringkey,Data_Items_4_m_Item_stringkey,Data_Items_5_m_Item_stringkey,Data_Items_6_m_Item_stringkey,Data_Items_7_m_Item_stringkey,Data_Items_8_m_Item_stringkey,Data_Items_9_m_Item_stringkey,Data_Items_0_m_Item_text,Data_Items_1_m_Item_text,Data_Items_2_m_Item_text,Data_Items_3_m_Item_text,Data_Items_4_m_Item_text,Data_Items_5_m_Item_text,Data_Items_6_m_Item_text,Data_Items_7_m_Item_text,Data_Items_8_m_Item_text,Data_Items_9_m_Item_text,All_Items,All_Item_Ids
0,0,919118b9d72840dabef59fb9619a8b20,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,AmourCourtoisBook,False,1.0,2841782e80bc48b1927677b3a88e392e,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,66b9351d457f4363afebcbca96a506da,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_AmourCourtoisBook.jbp,Upper Decks,Верхние палубы,bfc73060-7be4-425c-a2c2-4f0ff2d287c2,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,,,,,,,,,,Secrets of Gallant Conduct: How to Express Your Feelings to a Noble Lady,2841782e80bc48b1927677b3a88e392e
1,1,136d2833bd834cdab8cb0e8b65ea1bdc,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,GoodLootHidden,False,,,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,54b22e52a6da46eaa8c0f1fe2049d040,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_GoodLootHidden.jbp,Ruined Palace,Разрушенный дворец,,,,,,,,,,,,,,,,,,,,,,
2,2,0fe886222a234ad8abd7022a2d6ef04f,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,Loot4,False,1.0,199b3d86b5214c0c8fe483806f3122ce,3.0,edf1ca959e0f40f4bf5bb64586f816b9,1.0,04292c838432460396cfce58af861338,1.0,e29803f0fb444c90a153a2563cf30ae7,1.0,e239a9c0eaa84444a12b5aff054a0085,,,,,,,,,,,AdeptusMechanicus,Good,5abaf24cde1f4674a658da370195f537,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_Loot4.jbp,Outer Spire Halls,Внешние палаты шпиля,ce572ed4-94db-4ebf-8762-ccbb5671c404,45090443-c462-4cc5-8bef-f7d269a0bbdc,6d2d536e-77ac-4fd6-87c9-a2eb9a8f5cd1,0895bcbb-c5b8-4049-b516-859215d8735c,4839b9f3-2ed5-4a16-b895-a6bf99c21a88,,,,,,Armour of Despair,Xenospasm Grenade,Avenger Tactical Gauntlets,Peripheral Control Monocle,Melta Charge,,,,,,"Armour of Despair, Xenospasm Grenade, Avenger Tactical Gauntlets, Peripheral Control Monocle, Melta Charge","199b3d86b5214c0c8fe483806f3122ce, edf1ca959e0f40f4bf5bb64586f816b9, 04292c838432460396cfce58af861338, e29803f0fb444c90a153a2563cf30ae7, e239a9c0eaa84444a12b5aff054a0085"
3,3,a613e4a00a66493a82eb9609ecb8cc61,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (3),False,1.0,f123c305357a44f58b1d6449ded7cda7,1.0,8d9ec005c04a4984ad6032b3f74d84b1,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Loot\Good\CR0\AdeptusMechanicus_LootGood (3).jbp,Upper Decks,Верхние палубы,e9a5f200-a460-4f0d-ae4e-69a58468d2d7,ff264714-1839-493e-90f0-b48cd7c9976c,,,,,,,,,Master Surgeon Gloves,Advanced Medikit,,,,,,,,,"Master Surgeon Gloves, Advanced Medikit","f123c305357a44f58b1d6449ded7cda7, 8d9ec005c04a4984ad6032b3f74d84b1"
4,4,aed888571d4c4e69b5e063e36ddddfb5,"0449d0493fd70da4ba79ef76be174b92, BlueprintLoot",FullCrate,LootGood (5),False,1.0,60989b213e8547799e6c17ed9b0238c0,,,,,,,,,,,,,,,,,,,AdeptusMechanicus,Good,a0f840a0a35043a19bec5cc116f154cf,e:\Steam\RogueTrader\WorkFolder\Blueprints\Lo

In [248]:
def calculate_item_location(map_df, #dataframe to merge with  
                             df, #dataframe calculate location column for
                             columns # list of columns search through in loot table
                             ):
  """ Function to calculate item location column by searching through loot table columns with items ids  """
  created_columns = [] #list of output columns    
  #uniques_df = equip_items_df.query('Data_m_Rarity == "Unique"')
  uniques_df = map_df.copy(deep=True) #dataframe needed for final mapping values
  for i in range(len(columns)):
      """for loop for creating separate location column for every column in incoming list of columns"""
      temp_df = loot_table[['Data_Items_{}_m_Item'.format(i),
                          'Area_Name','Area_Name_Ru',
                            'Data_Items_{}_m_Item_text'.format(i)]
                            ].merge(
                                    map_df,
                                    how = 'right',
                                    left_on = ['Data_Items_{}_m_Item'.format(i),'Data_Items_{}_m_Item_text'.format(i)],
                                    right_on=['AssetId','Item_Name'],
                                    indicator='exists'
                                    )
      map_dict = dict(zip(temp_df['Item_Name'],temp_df['Area_Name']))
      uniques_df['Item_Location_{}'.format(i)] = uniques_df['Item_Name'].map(map_dict)
      created_columns.append('Item_Location_{}'.format(i)) #add newly created columns to list for further use

  uniques_df[created_columns]= uniques_df[created_columns].fillna('') #fill nans for empty rows
  #Concatenate strings across output columns to get final location or list of locations
  uniques_df['Item_Location'] = uniques_df[created_columns].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
  #create column with item location in the main equip table
  final_map_dict = dict(zip(uniques_df['AssetId'],uniques_df['Item_Location']))
  df['Item_Location'] = df['AssetId'].map(final_map_dict).fillna('')
  return df

In [249]:
## list of columns to search items ids through in loot_table
columns = find_item_columns(loot_table)
# calculate armor locations
armors_df = calculate_item_location(map_df = armors_df[['AssetId','Item_Name',]],
                                    df = armors_df,
                                    columns = columns)
#calculate locations for equip table
equip_items_df = calculate_item_location(map_df = equip_items_df[['AssetId','Item_Name']],
                                    df = equip_items_df,
                                    columns = columns)
#calculate location for weapons
weapons_table = calculate_item_location(map_df = weapons_table[['AssetId','Item_Name']],
                                    df = weapons_table,
                                    columns = columns)


In [250]:
equip_items_df.head()

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Vendor_Name,Is_Vendor,Item_Location
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter2,True,
1,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True,
2,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True,
3,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter4,True,
4,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,KasballicaMissionChapter2,True,


Add item location for russian version:

In [251]:
weapons_table_ru.head(2)

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет
1,cd5fa172af484872820ffbb0d713985b,[Модель Мезоа] Электродуговое ружье,При попадании в цель из электродугового ружья вы также атакуете всех противников в радиусе 2 клеток от цели.,,Сплошное,Нет,Двуручное,RangedWeaponry,Pattern,Электрический,0.0,1.0,13.0,20.0,4.0,18.0,Ranged,30.0,10.0,10.0,One,Одиночный выстрел,Одиночный выстрел из используемого дистанционного оружия.,2.0,SingleShot,Перезарядка,Перезаряжает выбранное оружие.,3.0,Reload,,,0.0,,,,0.0,,,,0.0,,Эксплораторы_Глава_2,Да


In [252]:
def calculate_item_location_ru(map_df, #dataframe to merge with  
                             df, #dataframe calculate location column for
                             columns # list of columns search through in loot table
                             ):
  """ Function to calculate item location column by searching through loot table columns with items ids  """
  created_columns = [] #list of output columns    
  #uniques_df = equip_items_df.query('Data_m_Rarity == "Unique"')
  uniques_df = map_df.copy(deep=True) #dataframe needed for final mapping values
  for i in range(len(columns)):
      """for loop for creating separate location column for every column in incoming list of columns"""
      temp_df = loot_table_ru[['Data_Items_{}_m_Item'.format(i),
                          'Area_Name','Area_Name_Ru',
                            'Data_Items_{}_m_Item_text'.format(i)]
                            ].merge(
                                    map_df,
                                    how = 'right',
                                    left_on = ['Data_Items_{}_m_Item'.format(i),'Data_Items_{}_m_Item_text'.format(i)],
                                    right_on=['AssetId','Item_Name'],
                                    indicator='exists'
                                    )
      map_dict = dict(zip(temp_df['Item_Name'],temp_df['Area_Name_Ru']))
      uniques_df['Item_Location_{}'.format(i)] = uniques_df['Item_Name'].map(map_dict)
      created_columns.append('Item_Location_{}'.format(i)) #add newly created columns to list for further use

  uniques_df[created_columns]= uniques_df[created_columns].fillna('') #fill nans for empty rows
  #Concatenate strings across output columns to get final location or list of locations
  uniques_df['Item_Location'] = uniques_df[created_columns].astype(str).agg(lambda x: ', '.join(word for word in x if word.strip()), axis = 1)
  #create column with item location in the main equip table
  final_map_dict = dict(zip(uniques_df['AssetId'],uniques_df['Item_Location']))
  df['Item_Location'] = df['AssetId'].map(final_map_dict).fillna('')
  return df

In [253]:
armors_df_ru[['Item_Name','Item_Name_Ru']]

,Item_Name,Item_Name_Ru
0,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,
1,Рунический доспех,
2,Улучшенный бронированный гермокостюм,
3,Надежный бронированный гермокостюм,
4,Усиленный бронированный гермокостюм,
...,...,...
98,Силовая броня Сороритас,
99,Силовая броня Сороритас,
100,Пропитанная ядом кольчуга,
101,Одеяние верного,


In [254]:
#Rename column to function work
weapons_table_ru = weapons_table_ru.rename(columns = {'Название':'Item_Name'})

In [255]:
## list of columns to search items ids through in loot_table
columns = find_item_columns(loot_table_ru)
# calculate armor locations
armors_df_ru = calculate_item_location_ru(map_df = armors_df_ru[['AssetId','Item_Name',]],
                                    df = armors_df_ru,
                                    columns = columns)

weapons_table_ru = calculate_item_location_ru(map_df = weapons_table_ru[['AssetId','Item_Name',]],
                                    df = weapons_table_ru,
                                    columns = columns).rename(columns = {'Item_Name':'Название'})

equip_items_df_ru = calculate_item_location_ru(map_df = equip_items_df_ru[['AssetId','Item_Name',]],
                                    df = equip_items_df_ru,
                                    columns = columns)

In [256]:
display(weapons_table[['AssetId','Item_Name','Item_Location']].head())
display(weapons_table_ru[['AssetId','Название','Item_Location']].head())

,AssetId,Item_Name,Item_Location
0,24db44debaed469386fad90088c20ee9,Ancient Force Sword,Unidentified Ruins
1,cd5fa172af484872820ffbb0d713985b,[Mezoa-Pattern] Arc Rifle,
2,bf9d592fc8444f29a99e27486182afb6,Alpha's Arc Rifle,
3,89922abe7a3b4a1d9359ba7d316114fc,Arc Rifle,Electrodynamic Cenobium
4,0b1a6f7920114dafb5ffc2bdea37bf8d,Astartes Bolter,


,AssetId,Название,Item_Location
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Неизвестные руины
1,cd5fa172af484872820ffbb0d713985b,[Модель Мезоа] Электродуговое ружье,
2,bf9d592fc8444f29a99e27486182afb6,Электродуговое ружье альфы,
3,89922abe7a3b4a1d9359ba7d316114fc,Электродуговое ружье,Электродинамическая обитель
4,0b1a6f7920114dafb5ffc2bdea37bf8d,Болтер Астартес,


#### Add project items to armor, equipment and weapons tables

English version, armor table:

In [257]:
#display(armors_df.head(1))
#display(project_items_en.head(1))

In [258]:
armors_df = armors_df.merge(project_items_en[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)

In [259]:
armors_df['Project_Item'] =  'Planet: '+ armors_df['Planet_Name'] +', ' + 'project: ' + armors_df['Project_Name']
# if Item_Location is empty put value of Project_item there:
armors_df['Item_Location'] = np.where(armors_df['Item_Location']=="", armors_df['Project_Item'], armors_df['Item_Location'])
armors_df = armors_df.fillna('')

In [260]:
# Create bool column to select only project items
armors_df.loc[armors_df['Project_Item']!="",'Is_Project_Item'] = True
armors_df.loc[armors_df['Project_Item']=="",'Is_Project_Item'] = False

Russian version, armor table:

In [261]:
armors_df_ru = armors_df_ru.merge(project_items_ru[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)
#Create project column
armors_df_ru['Project_Item'] =  'Планета: '+ armors_df_ru['Planet_Name'] +', ' + 'проект: ' + armors_df_ru['Project_Name']
# if Item_Location is empty put value of Project_item there:
armors_df_ru['Item_Location'] = np.where(armors_df_ru['Item_Location']=="", armors_df_ru['Project_Item'], armors_df_ru['Item_Location'])
armors_df_ru = armors_df_ru.fillna('')
# Create bool column to select only project items
armors_df_ru.loc[armors_df_ru['Project_Item']!="",'Is_Project_Item'] = 'Да'
armors_df_ru.loc[armors_df_ru['Project_Item']=="",'Is_Project_Item'] = 'Нет'

In [262]:
display(armors_df_ru.query('Project_Name.notna()').head(1))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
0,9c163228266c4cc892dfb98086d27b52,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,35f5651a088d4824951002d4910922fd,,98b410bd49bd4b74b00c78b5b4fba32f,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",3177d0cf74384129877de82675c5355e,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,,,,,,,,,,,,,,,,,,,,70745aa6-22f8-4d76-bb74-868c49fe6b94,,fe562964-fdd5-4da2-9a2b-6071128e8647,,,Armours,Pattern,65.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldarGuardianMeshArmour_Item.jbp,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Базовое свойство: +15% к броне, если значение Ловкости больше 80.\nСвойство мира: +15 к Ловкости.",,,Medium,,0.0,,,None,Human,,,,False,,,,,Нет


**Add project items to weapons, english version**

In [263]:
weapons_table = weapons_table.merge(project_items_en[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)

In [264]:
weapons_table['Project_Item'] =  'Planet: '+ weapons_table['Planet_Name'] +', ' + 'project: ' + weapons_table['Project_Name']
# if Item_Location is empty put value of Project_item there:
weapons_table['Item_Location'] = np.where(weapons_table['Item_Location']=="", weapons_table['Project_Item'], weapons_table['Item_Location'])
weapons_table = weapons_table.fillna('')
# Create bool column to select only project items
weapons_table.loc[weapons_table['Project_Item']!="",'Is_Project_Item'] = True
weapons_table.loc[weapons_table['Project_Item']=="",'Is_Project_Item'] = False

In [265]:
# check if project weapon  is already present in item_location column
display(weapons_table.query('Project_Item!=""').head(14)[weapons_table.columns[-6:]])

,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
42,False,"Planet: Vheabos VI, project: Blood Sports",Vheabos VI,Blood Sports,"Planet: Vheabos VI, project: Blood Sports",True
69,False,"Planet: Vheabos VI, project: Navy Whips",Vheabos VI,Navy Whips,"Planet: Vheabos VI, project: Navy Whips",True
81,False,"Planet: Dargonus, project: Kasballican Auction",Dargonus,Kasballican Auction,"Planet: Dargonus, project: Kasballican Auction",True
172,False,"Planet: Vheabos VI, project: Repentance of the Lost",Vheabos VI,Repentance of the Lost,"Planet: Vheabos VI, project: Repentance of the Lost",True
321,False,"Planet: Foulstone, project: Shrine",Foulstone,Shrine,"Planet: Foulstone, project: Shrine",True
322,False,"Planet: Kiava Gamma, project: Blessed Are the Weaponsmiths",Kiava Gamma,Blessed Are the Weaponsmiths,"Planet: Kiava Gamma, project: Blessed Are the Weaponsmiths",True
332,False,"Planet: Dargonus, project: Society of Cartographers",Dargonus,Society of Cartographers,"Planet: Dargonus, project: Society of Cartographers",True
333,False,"Planet: Foulstone, project: Bonfires of Righteousness",Foulstone,Bonfires of Righteousness,"Planet: Foulstone, project: Bonfires of Righteousness",True
343,False,"Planet: Foulstone, project: For Saint Drusus!",Foulstone,For Saint Drusus!,"Planet: Foulstone, project: For Saint Drusus!",True
349,False,"Planet: Kiava Gamma, project: Calculating Holiness",Kiava Gamma,Calculating Holiness,"Planet: Kiava Gamma, project: Calculating Holiness",True


**Add project items to weapons, russian version**

In [266]:
weapons_table_ru.head(1)

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет,Item_Location
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет,Неизвестные руины


In [267]:
weapons_table_ru = weapons_table_ru.merge(project_items_ru[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)
weapons_table_ru['Награда_За_Проект'] =  'Планета: '+ weapons_table_ru['Planet_Name'] +', ' + 'проект: ' + weapons_table_ru['Project_Name']

weapons_table_ru = weapons_table_ru.fillna('')
# if Item_Location is empty put value of Project_item there:
weapons_table_ru['Item_Location'] = np.where(weapons_table_ru['Item_Location']=="", 
                                             weapons_table_ru['Награда_За_Проект'], weapons_table_ru['Item_Location'])
# Create bool column to select only project items
weapons_table_ru.loc[weapons_table_ru['Награда_За_Проект']!="",'Оружие_За_Проект'] = 'Да'
weapons_table_ru.loc[weapons_table_ru['Награда_За_Проект']=="",'Оружие_За_Проект'] = 'Нет'

In [268]:
weapons_table_ru.query('Planet_Name!=""')[weapons_table_ru.columns[-6:]]

,Фракционный_Предмет,Item_Location,Planet_Name,Project_Name,Награда_За_Проект,Оружие_За_Проект
42,Нет,"Планета: Веабос-6, проект: Кровавые развлечения",Веабос-6,Кровавые развлечения,"Планета: Веабос-6, проект: Кровавые развлечения",Да
69,Нет,"Планета: Веабос-6, проект: Кнуты Флота",Веабос-6,Кнуты Флота,"Планета: Веабос-6, проект: Кнуты Флота",Да
81,Нет,"Планета: Даргонус, проект: Аукцион Касбаллики",Даргонус,Аукцион Касбаллики,"Планета: Даргонус, проект: Аукцион Касбаллики",Да
172,Нет,"Планета: Веабос-6, проект: Покаяние заблудших",Веабос-6,Покаяние заблудших,"Планета: Веабос-6, проект: Покаяние заблудших",Да
321,Нет,"Планета: Фолстоун, проект: Святыня",Фолстоун,Святыня,"Планета: Фолстоун, проект: Святыня",Да
322,Нет,"Планета: Кьява Гамма, проект: Блаженны оружейники",Кьява Гамма,Блаженны оружейники,"Планета: Кьява Гамма, проект: Блаженны оружейники",Да
332,Нет,"Планета: Даргонус, проект: Общество картографистов",Даргонус,Общество картографистов,"Планета: Даргонус, проект: Общество картографистов",Да
333,Нет,"Планета: Фолстоун, проект: Костры праведности",Фолстоун,Костры праведности,"Планета: Фолстоун, проект: Костры праведности",Да
343,Нет,"Планета: Фолстоун, проект: За святого Друза!",Фолстоун,За святого Друза!,"Планета: Фолстоун, проект: За святого Друза!",Да
349,Нет,"Планета: Кьява Гамма, проект: Вычисляющие святость",Кьява Гамма,Вычисляющие святость,"Планета: Кьява Гамма, проект: Вычисляющие святость",Да


In [269]:
missing_values_tab(weapons_table_ru)

Dataframe contains 48 columns and 416 strings.

It has  0 columns with missed values


,missed_values,%_of_all,data_type


**Add project items to equip table**

In [270]:
display(equip_items_df.head(1))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Vendor_Name,Is_Vendor,Item_Location
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,DrusianChapter2,True,


In [271]:
equip_items_df = equip_items_df.merge(project_items_en[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)

equip_items_df['Project_Item'] =  'Planet: '+ equip_items_df['Planet_Name'] +', ' + 'project: ' + equip_items_df['Project_Name']
equip_items_df = equip_items_df.fillna('')

In [272]:
#Check how many items already have values in item_location
display(equip_items_df[(equip_items_df['Item_Location']!="") 
                       & (equip_items_df['Project_Item']!="")].shape[0])

0

In [273]:
# if Item_Location is empty put value of Project_item there:
equip_items_df['Item_Location'] = np.where(equip_items_df['Item_Location']=="", equip_items_df['Project_Item'], equip_items_df['Item_Location'])

# Create bool column to select only project items
equip_items_df.loc[equip_items_df['Project_Item']!="",'Is_Project_Item'] = True
equip_items_df.loc[equip_items_df['Project_Item']=="",'Is_Project_Item'] = False

In [274]:
equip_items_df.query('Project_Item!=""')[equip_items_df.columns[-6:]]

,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
85,False,"Planet: Dargonus, project: Adeptus Arbites",Dargonus,Adeptus Arbites,"Planet: Dargonus, project: Adeptus Arbites",True
119,False,"Planet: Dargonus, project: Schola Progenium",Dargonus,Schola Progenium,"Planet: Dargonus, project: Schola Progenium",True
209,False,"Planet: Foulstone, project: Holy Defenders",Foulstone,Holy Defenders,"Planet: Foulstone, project: Holy Defenders",True
217,False,"Planet: Dargonus, project: Librarium",Dargonus,Librarium,"Planet: Dargonus, project: Librarium",True
222,False,"Planet: Kiava Gamma, project: Sacrilege of Creation",Kiava Gamma,Sacrilege of Creation,"Planet: Kiava Gamma, project: Sacrilege of Creation",True
274,False,"Planet: Vheabos VI, project: Raider Base",Vheabos VI,Raider Base,"Planet: Vheabos VI, project: Raider Base",True
343,False,"Planet: Janus, project: The Science of Virtue",Janus,The Science of Virtue,"Planet: Janus, project: The Science of Virtue",True
349,False,"Planet: Kiava Gamma, project: Warp Revelations",Kiava Gamma,Warp Revelations,"Planet: Kiava Gamma, project: Warp Revelations",True
367,False,"Planet: Janus, project: Death Cult of the Reaper Emperor",Janus,Death Cult of the Reaper Emperor,"Planet: Janus, project: Death Cult of the Reaper Emperor",True
396,False,"Planet: Kiava Gamma, project: Restoration",Kiava Gamma,Restoration,"Planet: Kiava Gamma, project: Restoration",True


Russian version:

In [275]:
display(equip_items_df_ru.head(1))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_2,True,


In [276]:
project_items_en.head(1)

,AssetId,Planet_Name,Project_Rank,Project_Name,Item_Name,Item_AssetId
0,291b8f8b44504192834bdac694d1517b,Kiava Gamma,Fourth,Dark Experiments,"""Fangs of Janus"" Macro-Weapons",57818d38b6924b6ba972af645664ecbb


In [277]:
equip_items_df_ru = equip_items_df_ru.merge(project_items_ru[['Item_AssetId','Planet_Name','Project_Name']], how = 'left', 
                            left_on = ['AssetId'],
                              right_on = ['Item_AssetId'],
                              suffixes=('_x','_y')).drop(columns=['Item_AssetId'], axis = 1)
equip_items_df_ru = equip_items_df_ru.fillna('')

In [278]:
display(equip_items_df_ru.head(1))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_2,True,,,


In [279]:
#equip_items_df_ru['Project_Item'] =  'Планета: '+ equip_items_df_ru['Planet_Name'] +', ' + 'проект: ' + equip_items_df_ru['Project_Name']
equip_items_df_ru.loc[equip_items_df_ru['Planet_Name']!="",'Project_Item'] = 'Планета: '+ equip_items_df_ru['Planet_Name'] +', ' + 'проект: ' + equip_items_df_ru['Project_Name']

In [280]:
# if Item_Location is empty put value of Project_item there:
equip_items_df_ru['Item_Location'] = np.where(equip_items_df_ru['Item_Location']=="", 
                                             equip_items_df_ru['Project_Item'], equip_items_df_ru['Item_Location'])
equip_items_df_ru = equip_items_df_ru.fillna('')


In [281]:
# Create bool column to select only project items
equip_items_df_ru.loc[equip_items_df_ru['Project_Item']!="",'Is_Project_Item'] = 'Да'
equip_items_df_ru.loc[equip_items_df_ru['Project_Item']=="",'Is_Project_Item'] = 'Нет'

In [282]:
display(equip_items_df_ru.query('Project_Item !=""')[equip_items_df_ru.columns[-6:]])


,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
85,False,"Планета: Даргонус, проект: Адептус Арбайтес",Даргонус,Адептус Арбайтес,"Планета: Даргонус, проект: Адептус Арбайтес",Да
119,False,"Планета: Даргонус, проект: Скола Прогениум",Даргонус,Скола Прогениум,"Планета: Даргонус, проект: Скола Прогениум",Да
209,False,"Планета: Фолстоун, проект: Святые защитники",Фолстоун,Святые защитники,"Планета: Фолстоун, проект: Святые защитники",Да
217,False,"Планета: Даргонус, проект: Либрариум",Даргонус,Либрариум,"Планета: Даргонус, проект: Либрариум",Да
222,False,"Планета: Кьява Гамма, проект: Грех сотворения",Кьява Гамма,Грех сотворения,"Планета: Кьява Гамма, проект: Грех сотворения",Да
274,False,"Планета: Веабос-6, проект: Рейдерская база",Веабос-6,Рейдерская база,"Планета: Веабос-6, проект: Рейдерская база",Да
343,False,"Планета: Янус, проект: Наука праведности",Янус,Наука праведности,"Планета: Янус, проект: Наука праведности",Да
349,False,"Планета: Кьява Гамма, проект: Варп-откровения",Кьява Гамма,Варп-откровения,"Планета: Кьява Гамма, проект: Варп-откровения",Да
367,False,"Планета: Янус, проект: Культ Смерти Императора Пожинающего",Янус,Культ Смерти Императора Пожинающего,"Планета: Янус, проект: Культ Смерти Императора Пожинающего",Да
396,False,"Планета: Кьява Гамма, проект: Реставрация",Кьява Гамма,Реставрация,"Планета: Кьява Гамма, проект: Реставрация",Да


In [283]:
equip_items_df_ru['Project_Item'].unique()

array(['', 'Планета: Даргонус, проект: Адептус Арбайтес',
       'Планета: Даргонус, проект: Скола Прогениум',
       'Планета: Фолстоун, проект: Святые защитники',
       'Планета: Даргонус, проект: Либрариум',
       'Планета: Кьява Гамма, проект: Грех сотворения',
       'Планета: Веабос-6, проект: Рейдерская база',
       'Планета: Янус, проект: Наука праведности',
       'Планета: Кьява Гамма, проект: Варп-откровения',
       'Планета: Янус, проект: Культ Смерти Императора Пожинающего',
       'Планета: Кьява Гамма, проект: Реставрация',
       'Планета: Янус, проект: Ксенооборона',
       'Планета: Янус, проект: Заблудшие',
       'Планета: Веабос-6, проект: Священная каторга',
       'Планета: Фолстоун, проект: Идущие за Друзом',
       'Планета: Кьява Гамма, проект: Движущая сила есть жизнь',
       'Планета: Фолстоун, проект: Маглев-дороги'], dtype=object)

In [284]:
equip_items_df_ru.head(1)

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_2,True,,,,,Нет


#### Add dialog items

##### Armors, english version

In [285]:
display(dialog_items_table_eng.head(2))
display(armors_df.head(1))
print(dialog_items_table_eng['Planet_Name'].unique())


,Item_AssetId,Item_Name,Planet_Name,Dialog_Item_Location
0,2612a899accd40faa1639661c2a1b7f0,This ancient component of a Dargonus-fleet frigate was discovered in the wastes by the Engine Vandals and presented to the Rogue Trader.\nTorpedo tubes capable of firing 5 short-burning plasma torpedoes with 19 damage warheads.,Dargonus,"Dargonus, construction events"
1,6e851ded713b408aae6d2a64d5a1f4ae,Wasteland Twister,Dargonus,"Dargonus, construction events"


,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Vendor_Name,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
0,9c163228266c4cc892dfb98086d27b52,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,35f5651a088d4824951002d4910922fd,,98b410bd49bd4b74b00c78b5b4fba32f,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",3177d0cf74384129877de82675c5355e,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,,,,,,,,,,,,,,,,,,,,70745aa6-22f8-4d76-bb74-868c49fe6b94,,fe562964-fdd5-4da2-9a2b-6071128e8647,,,Armours,Pattern,65.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldarGuardianMeshArmour_Item.jbp,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,,Medium,,0.0,,,None,Human,,,False,,,,,False


['Dargonus' 'Foulstone' 'Janus' 'Kiava Gamma' 'Vheabos VI']


In [286]:
armors_df = armors_df.merge(dialog_items_table_eng[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
armors_df['Item_Location'] = np.where(armors_df['Item_Location']=="", armors_df['Dialog_Item_Location'], armors_df['Item_Location'])
armors_df.loc[armors_df['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = True
armors_df.loc[armors_df['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = False
armors_df = armors_df.fillna('')                                                   

In [287]:
display(armors_df[armors_df['Dialog_Item_Location']!=''])

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Vendor_Name,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item,Dialog_Item_Location,Is_ConstructionEvent_Item
94,9e129aa1b6244db7b567ee028247d2ec,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,e57f59c271872dd4da2de66c6feb5fc5,565d10310f1a4a7c9fc7580d69eb2e8c,0830b1552ae62c74c812aef2a41edd1a,68287a54721b4dbe9ddc9a2e4ce9e2ee,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",49cb4eedad8043ceb3947b136abf6e25,,,,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",8c72b79d4b5d4d018cab39e075a40365,,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,590cb5c394ee4b5eb213121c96063dec,c7a53b43ac5f4f11a1aef612dd3f4f71,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",684c766dc9c24fcf856dc1d6f01d9290,,,,,,,,,,157702eb-57d5-4d1f-8abf-40f4c3561638,c7fe8afe-3401-4618-87c6-57b6cc15ec61,,ee307a53-154c-4f51-b9c0-4ea8d66e22f3,,Armours,Unique,55.0,3.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Unique\ReinforcedEnforcerCarapace_Item.jbp,Armour of Refraction,This carapace grants the wearer +1 deflection and +10% armour if the wearer has moved during their turn.\nBase armour property: +2 deflection against human enemies.,Carapace armour whose plates conceal miniature refractor generators that dissipate the energy of impacts. Blueprints for these generators are not contained in the catalogues of the Adeptus Mechanicus.,,Heavy,,0.0,,,Heavy,Human,,,False,"Kiava Gamma, construction events",,,,False,"Kiava Gamma, construction events",True


##### Armors, russian version

In [288]:
armors_df_ru = armors_df_ru.merge(dialog_items_table_ru[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
armors_df_ru['Item_Location'] = np.where(armors_df_ru['Item_Location']=="", armors_df_ru['Dialog_Item_Location'], armors_df_ru['Item_Location'])
armors_df_ru.loc[armors_df_ru['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = 'Да'
armors_df_ru.loc[armors_df_ru['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = 'Нет'
armors_df_ru = armors_df_ru.fillna('')       

In [289]:
armors_df_ru.query('Dialog_Item_Location!=""')[['Item_Name','Planet_Name','Item_Location','Dialog_Item_Location']].head()

,Item_Name,Planet_Name,Item_Location,Dialog_Item_Location
94,Панцирь рефракции,,"Кьява Гамма, строительство проектов","Кьява Гамма, строительство проектов"


##### Equipment table, English version

In [290]:
equip_items_df = equip_items_df.merge(dialog_items_table_eng[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
equip_items_df['Item_Location'] = np.where(equip_items_df['Item_Location']=="", equip_items_df['Dialog_Item_Location'], equip_items_df['Item_Location'])
#Create boolean column for the items
equip_items_df.loc[equip_items_df['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = True
equip_items_df.loc[equip_items_df['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = False
equip_items_df = equip_items_df.fillna('')  

In [291]:
equip_items_df.query('Dialog_Item_Location!=""')[['Item_Name','Planet_Name','Item_Location','Dialog_Item_Location']].head()

,Item_Name,Planet_Name,Item_Location,Dialog_Item_Location
112,Armour of Refraction,,"Kiava Gamma, construction events","Kiava Gamma, construction events"
394,Psychic Ward,,"Dargonus, construction events","Dargonus, construction events"
448,Book of Minor Prayers,,"Dargonus, construction events","Dargonus, construction events"
451,Cogni-Simulacrum EF-Ultima-Octodecimum,,"Kiava Gamma, construction events","Kiava Gamma, construction events"
458,Empyrean Transceiver,,"Kiava Gamma, construction events","Kiava Gamma, construction events"


##### equipment table, Russian version

In [292]:
display(equip_items_df_ru.head(1))

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
0,00eb6f0ff2e24009a4e6737ec50ff01d,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,,,0fa0782686d949a389eefe83f63e4658,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,None,Common,0.0,0.0,10.0,15.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\Mobs\GuardFlakArmourStandart_Item_Wrong.jbp,,,,,,Друзианцы_Глава_2,True,,,,,Нет


In [293]:
equip_items_df_ru = equip_items_df_ru.merge(dialog_items_table_ru[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
equip_items_df_ru['Item_Location'] = np.where(equip_items_df_ru['Item_Location']=="", equip_items_df_ru['Dialog_Item_Location'],
                                                 equip_items_df_ru['Item_Location'])
#Create boolean column for the items
equip_items_df_ru.loc[equip_items_df_ru['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = 'Да'
equip_items_df_ru.loc[equip_items_df_ru['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = 'Нет'
equip_items_df_ru = equip_items_df_ru.fillna('') 

In [294]:
equip_items_df_ru.query('Dialog_Item_Location==""')[['Item_Name','Planet_Name',
'Item_Location','Dialog_Item_Location','Is_ConstructionEvent_Item']].head()

,Item_Name,Planet_Name,Item_Location,Dialog_Item_Location,Is_ConstructionEvent_Item
0,,,,,Нет
1,,,,,Нет
2,,,,,Нет
3,,,,,Нет
4,,,,,Нет


##### Weapons, English version

In [295]:
weapons_table.head(1)

,AssetId,Item_Name,Item_Descr,Add_Descr,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Cargo_Volume_Percent,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Vendor_Name,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item
0,24db44debaed469386fad90088c20ee9,Ancient Force Sword,This sword deals an additional +psy rating damage.,,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,15,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,,,0,,,,0,,,False,Unidentified Ruins,,,,False


In [296]:
weapons_table = weapons_table.merge(dialog_items_table_eng[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
weapons_table['Item_Location'] = np.where(weapons_table['Item_Location']=="", weapons_table['Dialog_Item_Location'], 
                                            weapons_table['Item_Location'])
#Create boolean column for the items
weapons_table.loc[weapons_table['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = True
weapons_table.loc[weapons_table['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = False
weapons_table = weapons_table.fillna('') 

In [297]:
weapons_table.query('Dialog_Item_Location==""')[['Item_Name','Planet_Name',
'Item_Location','Dialog_Item_Location','Is_ConstructionEvent_Item']].head()

,Item_Name,Planet_Name,Item_Location,Dialog_Item_Location,Is_ConstructionEvent_Item
0,Ancient Force Sword,,Unidentified Ruins,,False
1,[Mezoa-Pattern] Arc Rifle,,,,False
2,Alpha's Arc Rifle,,,,False
3,Arc Rifle,,Electrodynamic Cenobium,,False
4,Astartes Bolter,,,,False


##### Weapons, Russian version

In [298]:
display(weapons_table_ru.head(1))

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет,Item_Location,Planet_Name,Project_Name,Награда_За_Проект,Оружие_За_Проект
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет,Неизвестные руины,,,,Нет


In [299]:
weapons_table_ru = weapons_table_ru.merge(dialog_items_table_ru[['Item_AssetId','Dialog_Item_Location']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_AssetId'],
                                                    suffixes = (None,'_diDF')).drop(columns = ['Item_AssetId'], axis = 1)
# Add item location if cell is empty
weapons_table_ru['Item_Location'] = np.where(weapons_table_ru['Item_Location']=="", weapons_table_ru['Dialog_Item_Location'], 
                                            weapons_table_ru['Item_Location'])
#Create boolean column for the items
weapons_table_ru.loc[weapons_table_ru['Dialog_Item_Location'].notna(), 'Is_ConstructionEvent_Item'] = 'Да'
weapons_table_ru.loc[weapons_table_ru['Dialog_Item_Location'].isna(), 'Is_ConstructionEvent_Item'] = 'Нет'
weapons_table_ru = weapons_table_ru.fillna('') 

In [300]:
weapons_table_ru.query('Dialog_Item_Location!=""')[['Название','Planet_Name',
'Item_Location','Dialog_Item_Location','Is_ConstructionEvent_Item']].head()

,Название,Planet_Name,Item_Location,Dialog_Item_Location,Is_ConstructionEvent_Item
171,Буря Пустошей,,"Даргонус, строительство проектов","Даргонус, строительство проектов",Да
226,Триумф Веры,,"Фолстоун, строительство проектов","Фолстоун, строительство проектов",Да
356,Правосудие Праведников,,"Фолстоун, строительство проектов","Фолстоун, строительство проектов",Да


#### Add iteam_areas_df

**Notes**:  
```item_areas_df``` will be merged with other tables only for unique items as they have only one location.  
Before that i should check how unique ids are distributed for each type of equipment to avoid duplicates.

In [301]:
display(item_areas_df.head())

,Item_Id,Area_Name,Area_Name_Ru
0,2841782e80bc48b1927677b3a88e392e,Upper Decks,Верхние палубы
1,199b3d86b5214c0c8fe483806f3122ce,Outer Spire Halls,Внешние палаты шпиля
2,f123c305357a44f58b1d6449ded7cda7,Upper Decks,Верхние палубы
3,60989b213e8547799e6c17ed9b0238c0,Upper Decks,Верхние палубы
4,f08a955b6c154086bd3c1b1445ef211a,Upper Decks,Верхние палубы


In [302]:
print(item_areas_df['Item_Id'].value_counts())

e239a9c0eaa84444a12b5aff054a0085    21
a01bd2e8da3e4a78805e92cde04419a0    16
56e3576b84384495b9015d5ec88217ac    12
0895828ef19a4245aaeca3335dd933de     7
0088d2c1ea084c1428266b7ffdeb6ab1     6
                                    ..
2802cf103af344f8b9d780ae4fb74a25     1
12d75eca428e4b6f9b43d16d35b46ea2     1
d2fcdc1b825044e3862e4be02769d694     1
10ad84f2802347feb90c93c79486308e     1
9050161424de439392bd6710898e0e4b     1
Name: Item_Id, Length: 312, dtype: int64


In [303]:
#create slice of unique items
armors_unique_df = armors_df.query('Data_m_Rarity=="Unique"')
list_of_unique_ids = armors_unique_df['AssetId'].unique().tolist() 
#check how many location unique armors have
#slice item_areas_df for unique armors os we can use it further for merging
item_areas_armors_unique = item_areas_df[item_areas_df['Item_Id'].isin(list_of_unique_ids)]
print(item_areas_armors_unique['Item_Id'].value_counts().sort_values(ascending=False))
#display(armors_unique_df.head())

f4371a530e0842558ec87364533a298e    1
e3b05e8fa1964910a4fe123578bbb941    1
c10b7c8d1f384b20849e3addc303667f    1
0f66a6db32fa45deb4c1edd7babad5f9    1
95acee28d8d9421c8a7b6b4c6ac53c67    1
278488aee97649cf8b3ecf26a57832ac    1
199b3d86b5214c0c8fe483806f3122ce    1
1bd833cbf6724268a92421f66ec657bf    1
ddd6d0a5bdfc4fefaed5f247cf46e852    1
6d0c57c9db824f1aa27828fcdf2f65e5    1
Name: Item_Id, dtype: int64


In [304]:
#create slice of unique equip items
equip_unique_df = equip_items_df.query('Data_m_Rarity=="Unique"')
list_of_unique_ids = equip_unique_df['AssetId'].unique().tolist() 
#check how many location unique armors have
item_areas_equip_unique = item_areas_df[item_areas_df['Item_Id'].isin(list_of_unique_ids)]
print(item_areas_equip_unique['Item_Id'].value_counts().sort_values(ascending=False))
#display(armors_unique_df.head())

e6f0df4ea4404b82821e98795f52ffb7    1
95acee28d8d9421c8a7b6b4c6ac53c67    1
16b906b8913740158610d359206679bf    1
a37fed113c5b4b3ca3f739e60ba82639    1
9718460b999547c3b6f830cd8340e88d    1
                                   ..
278488aee97649cf8b3ecf26a57832ac    1
0ddc7e3a842e459b80cb7e8511fe309e    1
d1e2fd184d2f424393dbcdc919b95088    1
40d7240079af4473baa1f1183b7d3bc7    1
6860829c87dd4af8a93d0a49a8c346c1    1
Name: Item_Id, Length: 73, dtype: int64


In [305]:
#create slice of unique weapons items
weapons_unique_df = weapons_table.query('Rarity=="Unique"')
list_of_unique_ids = weapons_unique_df['AssetId'].unique().tolist() 
#check how many location unique armors have
item_areas_weapons_unique = item_areas_df[item_areas_df['Item_Id'].isin(list_of_unique_ids)]
print(item_areas_weapons_unique['Item_Id'].value_counts().sort_values(ascending=False))
#display(armors_unique_df.head())

1e8a09ba6138475c9f105761ba245afb    1
0a26bcf222714df8bd54fc9c9792a258    1
5e536199ce074052babb98226bbd29fc    1
f75f6f41fdc74a7ba6ce177964dbed4e    1
e0a0a605a35d4a69b40f27ecbd7b886b    1
9376156b879442d6a33f3c5bf71d7aef    1
7e3f2d1c720240b7a62d7a9d2b709ee1    1
6168e8d5d7c44a05b07911f38f5f5a60    1
e3178a0e77ff45bfa7ac0144570f6f38    1
8f0075ae61de45d38a572f8ffb43927a    1
754ac64896a24d5cb3e829725e3ff012    1
e5d24d0226c04e98abe895ad32b1be76    1
10ad84f2802347feb90c93c79486308e    1
49d53db2e8eb41ee9e985926fb30c8d6    1
c9d56611363b4a13a6012b500c7afa33    1
3983a821be1b4f2f912e1ef6b7d73e86    1
a47a20d8edf14cc7bed8dba67c57d85b    1
4d87435ddfa042269c1fe35df0430f8b    1
Name: Item_Id, dtype: int64


As all tables have one location for each unique item, merging operation can continue.

Armors:

In [306]:
#English version
armors_df = armors_df.merge(item_areas_armors_unique[['Item_Id','Area_Name']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
armors_df['Item_Location'] = np.where(armors_df['Item_Location']=="", armors_df['Area_Name'], armors_df['Item_Location'])
armors_df = armors_df.fillna('') 

#Russian version
armors_df_ru = armors_df_ru.merge(item_areas_armors_unique[['Item_Id','Area_Name_Ru']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
armors_df_ru['Item_Location'] = np.where(armors_df_ru['Item_Location']=="", armors_df_ru['Area_Name_Ru'], armors_df_ru['Item_Location'])
armors_df_ru = armors_df_ru.fillna('') 

In [307]:
display(armors_df.query('Data_m_Rarity=="Unique"')[['Item_Name','Item_Location','Area_Name']].head())
display(armors_df_ru.query('Data_m_Rarity=="Unique"')[['Item_Name','Item_Location','Area_Name_Ru']].head())

,Item_Name,Item_Location,Area_Name
13,Harlequin Armour,,
49,Radiant Chestplate,,
64,Adaptive Flak Vest,Kiava Gamma Manufactorum,Kiava Gamma Manufactorum
65,Aeldari Farseer Mesh Armour,,
66,Aether-Rune Armour,,


,Item_Name,Item_Location,Area_Name_Ru
13,Доспех Арлекина,,
49,Сияющий нагрудник,,
64,Настраиваемый противоосколочный жилет,Мануфакторум Кьявы Гамма,Мануфакторум Кьявы Гамма
65,Плетеный доспех эльдарского провидца,,
66,Эфирно-рунический доспех,,


Equipment:

In [308]:
#English version
equip_items_df = equip_items_df.merge(item_areas_equip_unique[['Item_Id','Area_Name']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
equip_items_df['Item_Location'] = np.where(equip_items_df['Item_Location']=="", equip_items_df['Area_Name'], equip_items_df['Item_Location'])
equip_items_df = equip_items_df.fillna('') 

#Russian version
equip_items_df_ru = equip_items_df_ru.merge(item_areas_equip_unique[['Item_Id','Area_Name_Ru']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
equip_items_df_ru['Item_Location'] = np.where(equip_items_df_ru['Item_Location']=="", equip_items_df_ru['Area_Name_Ru'], equip_items_df_ru['Item_Location'])
equip_items_df_ru = equip_items_df_ru.fillna('') 

In [309]:
display(equip_items_df.query('Data_m_Rarity=="Unique"')[['Item_Name','Item_Location','Area_Name']].tail())
display(equip_items_df_ru.query('Data_m_Rarity=="Unique"')[['Item_Name','Item_Location','Area_Name_Ru']].tail())

,Item_Name,Item_Location,Area_Name
516,Transonic Emitter,"Planet: Foulstone, project: Maglev Railways",
517,Untuning Combi-Tool,Machine Cathedral,Machine Cathedral
518,Emerald Cherub Ring,,
519,Vid-Capture Relay,,
520,Volition Inhibitor,,


,Item_Name,Item_Location,Area_Name_Ru
516,Трансзвуковой излучатель,"Планета: Фолстоун, проект: Маглев-дороги",
517,Комби-тул разлада,Машинный храм,Машинный храм
518,Кольцо с изумрудным ангелом,,
519,Устройство для записи,,
520,Подавляющий ингибитор,,


Weapons:

In [310]:
#English version
weapons_table = weapons_table.merge(item_areas_weapons_unique[['Item_Id','Area_Name']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
weapons_table['Item_Location'] = np.where(weapons_table['Item_Location']=="", weapons_table['Area_Name'], weapons_table['Item_Location'])
weapons_table = weapons_table.fillna('') 

#Russian version
weapons_table_ru = weapons_table_ru.merge(item_areas_weapons_unique[['Item_Id','Area_Name_Ru']], how = 'left',
                                                    left_on = ['AssetId'], right_on = ['Item_Id'],
                                                    suffixes = (None,'_y')).drop(columns = ['Item_Id'], axis = 1)
# Add item location if cell is empty
weapons_table_ru['Item_Location'] = np.where(weapons_table_ru['Item_Location']=="", weapons_table_ru['Area_Name_Ru'], weapons_table_ru['Item_Location'])
weapons_table_ru = weapons_table_ru.fillna('') 

In [311]:
display(weapons_table.query('Rarity=="Unique"')[['Item_Name','Item_Location','Area_Name']].head())
display(weapons_table_ru.query('Редкость=="Unique"')[['Название','Item_Location','Area_Name_Ru']].head())

,Item_Name,Item_Location,Area_Name
47,Edge of the Irrevocable,,
50,Corrupted Inferno Pistol,Upper Decks,Upper Decks
70,[Kiava Gamma Pattern] Omnissian Axe,,
76,[Retobi-Pattern] Foehammer,,
79,Fractal Blade,Forests of Janus,Forests of Janus


,Название,Item_Location,Area_Name_Ru
47,Предел необратимости,,
50,"Оскверненный пистолет ""Инферно""",Верхние палубы,Верхние палубы
70,[Модель Кьява Гамма] Топор Омниссии,,
76,[Модель Ретоби] Крушитель Врагов,,
79,Фрактальный клинок,Леса Януса,Леса Януса


__________

In [312]:
item_areas_df.query('Item_Id=="24db44debaed469386fad90088c20ee9"')

,Item_Id,Area_Name,Area_Name_Ru
74,24db44debaed469386fad90088c20ee9,Unidentified Ruins,Неизвестные руины


In [313]:
weapons_table.head(2)

,AssetId,Item_Name,Item_Descr,Add_Descr,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Cargo_Volume_Percent,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Vendor_Name,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name
0,24db44debaed469386fad90088c20ee9,Ancient Force Sword,This sword deals an additional +psy rating damage.,,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,15,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,,,0,,,,0,,,False,Unidentified Ruins,,,,False,,False,
1,cd5fa172af484872820ffbb0d713985b,[Mezoa-Pattern] Arc Rifle,"When you hit the target with an arc rifle, you also attack any enemies within 2 cells of the target.",,Solid,False,TwoHanded,RangedWeaponry,Pattern,Shock,0,1,13,20,4,18,Ranged,30,10,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,,,0,,,,0,,,,0,,ExploratorChapter2,True,,,,,False,,False,


In [314]:
weapons_table_ru.head(2)

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет,Item_Location,Planet_Name,Project_Name,Награда_За_Проект,Оружие_За_Проект,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name_Ru
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет,Неизвестные руины,,,,Нет,,Нет,
1,cd5fa172af484872820ffbb0d713985b,[Модель Мезоа] Электродуговое ружье,При попадании в цель из электродугового ружья вы также атакуете всех противников в радиусе 2 клеток от цели.,,Сплошное,Нет,Двуручное,RangedWeaponry,Pattern,Электрический,0.0,1.0,13.0,20.0,4.0,18.0,Ranged,30.0,10.0,10.0,One,Одиночный выстрел,Одиночный выстрел из используемого дистанционного оружия.,2.0,SingleShot,Перезарядка,Перезаряжает выбранное оружие.,3.0,Reload,,,0.0,,,,0.0,,,,0.0,,Эксплораторы_Глава_2,Да,,,,,Нет,,Нет,


In [315]:
equip_items_df[equip_items_df['Data_$type'].str.contains("Gloves")]['Item_Location'].unique()

array(['', 'Drifting Voidship', 'Reaving Tempest Spire', 'Ruined Palace',
       'Machine Cathedral', 'Upper Decks', 'Ark Mechanicus Hermetico',
       'Von Valancius Palace', "Governor Vyatt's Estate",
       'Forests of Janus', 'Kiava Gamma Manufactorum', 'Smuggler Hideout',
       'Macro-Cannon Chamber', 'Outer Spire Halls',
       'Planet: Vheabos VI, project: Raider Base',
       'Footfall Shadow Quarters', 'Abandoned Landing Site', 'Chasm'],
      dtype=object)

In [316]:
equip_items_df[equip_items_df['Data_$type'].str.contains("Head")]['Item_Location'].unique()

array(['', 'Upper Decks', 'Unidentified Ruins',
       'Ark Mechanicus Hermetico', 'Machine Cathedral',
       'Plundered Void Station', 'Smuggler Hideout', 'Voidship Bridge',
       'The Upperway', 'Footfall Atrium', 'Outer Spire Halls', 'Chasm',
       'Electrodynamic Cenobium',
       'Planet: Janus, project: The Science of Virtue', 'Epitaph',
       'Drifting Voidship',
       'Planet: Kiava Gamma, project: Warp Revelations'], dtype=object)

###  Changes to data

The part is dedicated to adding changes, usually corrections to items' locations.

#### Dictionaries

In [317]:
items = {'Защита пальцев':'Атриум Футфола, канализация. Нужно проникнуть в крематорий на похоронах Беллардо.',
         'Шлем провокатора': 'Дрейфующий корабль, система Авиорус',
         'Эрго-сапоги':'Леса Януса, в сейфе к западу от места драки в низине',
         'Убийственная меткость':'Планета Грантис, система "Забытые близнецы", если успеть на сигнал о помощи.',
         'Доспех бесстрашия':'Планета Грантис, система "Забытые близнецы", если успеть на сигнал о помощи.',
         'Плащ с вышивкой':'Планета Грантис, система "Забытые близнецы", если успеть на сигнал о помощи.',
         'Имперская гордость':'Футфол, Теневые кварталы, убежище мутантов, после выполнения миссии на Янусе и доставки продовольствия.',
         'Набор реагентов':'Футфол, Теневые кварталы, убежище мутантов, после выполнения миссии на Янусе и доставки продовольствия.',
         'Сапоги пиромана':'Футфол, Теневые кварталы, внутри карантинной зоны.',
         'Нечестивый осколок':'Система "Столпы Виридики", корабль инквизиции.',
         'Панцирь Темпестус':'Система "Столпы Виридики", корабль инквизиции.',
         'Пистолет "Инферно"':'Система "Столпы Виридики", корабль инквизиции.',
         'Наперстный огнемет':'Мануфакторум Кьявы Гамма, хранилище, нужен код с мостика корабля в системе "Агатус Архидес".',
         'Кровавый посох':'Мануфакторум Кьявы Гамма, хранилище, нужен код с мостика корабля в системе "Агатус Архидес".',
         'Портативный когитатор':'Мануфакторум Кьявы Гамма.',
         '[Модель Мезоа] Плазма-ружье' :'Мануфакторум Кьявы Гамма.',
         'Визор рейнджера':'Разоренная станция, система "Пояс Лангренна".',
         'Побывавшая в бою накидка':'Разоренная станция, система "Пояс Лангренна".',
         'Шлем опытного стрелка':'Разоренная станция, система "Пояс Лангренна".',
         'Цепи преобладания':'Неизвестные руины, система "Пролив Латоти".',
         'Вуаль обманчивой прозорливости':'Неизвестные руины, система "Пролив Латоти".',
         'Игольная винтовка':'Система "Забытые Близнецы", мир джунглей, нужно предоставить провизию колонистам. Также в тайном убежище контрабандистов, глава 4.',
         'Археотечное лазружье':'Система "Лапидус Мортус", выжженый мир, вариант Джай.',
         'Усиленные сапоги':'Система "Лапидус Мортус", выжженый мир, вариант Джай.',
         'Инъектор со стимулятором':'Атриум Футфола, склад.',
         'Фрактальный клинок':'Система Ардемат II, стоянка реколлигеров.',
         'Хамелеолиновый плащ':'Система Ардемат II, стоянка реколлигеров.',
         'Плащ теней':'Система "Последний Шанс Кирены"',
         'Цепной меч "Скорпион"':'Система "Последний Шанс Кирены"',
         'Имперский посох':'Планета Даргонус, дворец фон Валанциус во время рейда друкари, нужна проверка Атлетики.',
         'Защитник веры':'Планета Даргонус, дворец фон Валанциус во время рейда друкари, нужно обезвредить мину с охранника.',
         'Сколовый дробовик':'Каморра, стать боссом Ямы.',
         'Осколок кровавого камня':'Каморра, стать боссом Ямы.',
         'Лазбластер':'Каморра, стать боссом Ямы.',
         'Цепной меч Астартес':'Каморра, стать боссом Ямы.',
         '[Модель Ретоби] Плазма-ружье':'Каморра, стать боссом Ямы.',
         'Перчатки опытного хирурга':'Верхние палубы, глава 3, квест Маражая.',
         'Визор архистратега':'Верхние палубы, глава 3, квест Маражая.',
         "Скрижали святого Когнация":"""Фолстоун, завершение проекта 3 ранга,
                                        проверка убеждения с пенальти -40.""", 
         "Элитный панцирный рюкзак Темпестус":"""Глава 4, Кетца-Темер, локация с трупом сенешаля Винтерскейла,
                                                 проверка техники с пенальти -80.""",
        "Цепной штык-нож":"Глава 4, Кетца-Темер.",
        "Горнило страданий":"Глава 4, Кетца-Темер.",
        "Ступор":  "Глава 4, Кетца-Темер.", 
        "Маскирующий плащ":  "Глава 4, Кетца-Темер.",   
        "Суин Келин":  "Глава 4, Кетца-Темер.",        
        "[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца":  "Глава 4, Кетца-Темер.", 
        "Плетеный доспех эльдарского провидца":  "Глава 4, Кетца-Темер.",
        "Накидка несущего рок":"Глава 4, Кетца-Темер, Лаборатория, проверка логики с пенальти -75.", 
        "Тиара Имматериума":"Глава 4, Кетца-Темер, Лаборатория.",    
        "Уловка Архимахинатора":"Глава 4, Кетца-Темер, Лаборатория.", 
        "Излучатель кровавого барьера":"Глава 4, Кетца-Темер, Лаборатория.",
        "Ведьмовской посох":"Глава 4, Кетца-Темер, Лаборатория.",  
        "Артефакт Гало":"Глава 4, Система Ничейные миры, заброшенный дворец.",
        "[Происхождение: мир-крепость] Панцирь Темпестус":"Глава 4, Система Ничейные миры, заброшенный дворец.",
        "Латные перчатки крестового похода":"Глава 4, Система Ничейные миры, заброшенный дворец.",
        "Кувалда костолома":"Глава 4, Система Ничейные миры, заброшенный дворец.",
        "Археотечное термоядерное ружье":"Глава 4, система Провидение Орселлио, Бесплодный мир, позволить Маражаю принять участие в резне.",
        "Плазма-кольцо":"Глава 4, система Смарагдус Мундус, квест Кассии.",
        "Рюкзак гренадера":"Глава 4, система Смарагдус Мундус, квест Кассии.",
        "Элитные перчатки стрелка":"Глава 4, система Смарагдус Мундус, квест Кассии.",
        "Неистовство Лилэтан":"""Планета Янус, завершение проекта 5 ранга,
        проверка наблюдательности с пенальти -95,
        Ирлиет должна быть в группе, реплика с упоминанием Ирлиет.""",
        "Плащ стремительной защиты":"Глава 4, система Несчастье Гартрафала, квест Хейнрикса",
        "Мантия подстрекателя":"Глава 4, система Несчастье Гартрафала, квест Хейнрикса",
        "Нечестивый фолиант":"Глава 4, система Несчастье Гартрафала, квест Хейнрикса",
        "Поганая филактерия":"Даргонус, завершение проекта 5 ранга и успех в любой проверке навыка.",
        
         

}
items_eng = {'Knuckle Guard':'Atrium Footfall, sewers. You need to infiltrate the crematorium at Bellardo''s funeral.',
         "Provocateur's Helmet": 'Drifting ship, "Aviorus system".',
         'Ergo Boots':'Forests of Janus, in a safe west of the fight in the valley',
         'Deadly Precision':'Planet Grantis, the system "Forgotten Twins", if you catch the signal for help.',
         'Armour of the Undaunted':'Planet Grantis, the system "Forgotten Twins", if you catch the signal for help.',
         'Embroidered Cloak':'Planet Grantis, the system "Forgotten Twins", if you catch the signal for help.',
         'Imperial Pride':'Footfall, Shadow Quarters, mutant hideout, after completing a mission on Janus and delivering food.',
         'Pack of Reagents':'Footfall, Shadow Quarters, mutant hideout, after completing a mission on Janus and delivering food.',
         'Pyromaniac Boots':'Foofall, Shadow Quaters, inside the restricted area.',
         'Unholy Shard':'The system "Pillars of Viridice", Drifting ship of the inquisition.',
         'Tempestus Carapace':'The system "Pillars of Viridice", Drifting ship of the inquisition.',
         'Inferno Pistol':'The system "Pillars of Viridice", Drifting ship of the inquisition.',
         'Flamer Digi-Weapon':'Kiava Gamma Manufactorum, storage, need the password from the ship''s bridge in the system "Agathus Archides".',
         'Staff of Blood':'Kiava Gamma Manufactorum, storage, need the password from the ship''s bridge in the system "Agathus Archides".',
         'Portable Cogitator':'Kiava Gamma Manufactorum.',
         '[Mezoa-Pattern] Plasma Gun' :'Kiava Gamma Manufactorum.',
         "Ranger's Visor":'Ravaged Void Station, the system "Langrenn''s Belt".',
         'Battle-Scarred Cape':'Ravaged Void Station, the system "Langrenn''s Belt".',
         "Gunslinger's Helmet":'Ravaged Void Station, the system "Langrenn''s Belt".',
         'Chains of Domination':'Unidentified Ruins, the system "Latotian''s Passage".',
         'Foulsight Veil':'Unidentified Ruins, the system "Latotian''s Passage".',
         'Needle Rifle':'The system "Forgotten Twins", jungle world, need to give food to colonists.Also in the Smuggler Hideout, chapter 4.',
         'Archeotech Lasgun':'The system "Lapidus Mortus", ash world, choice of Jae.',
         'Reinforced Boots':'The system "Lapidus Mortus", ash world, choice of Jae.',
         'Stimulant Injector':'Footfall,Atrium, warehouse.',
         'Fractal Blade':'The system "Ardematt II", Recolliger''s plot.',
         'Cameleoline Cloak':'The system "Ardematt II", Recolliger''s plot.',
         'Cloak of Shadows':'The system "The Last chance of Cyrene"',
         'Scorpion Chainsword':'The system "The Last chance of Cyrene"',
         'Imperial Staff':'The planet Dargonus, Von Valancius palace during drukari raid, need Athletics check.',
         'Protector of the Faith':'The planet Dargonus, Von Valancius palace during drukari raid, defuse mine from guard.',
         'Shard Shotgun':'Commorragh, become the boss of the Pit',
         'Shard of Bloodstone':'Commorragh, become the boss of the Pit.',
         'Lasblaster':'Commorragh, become the boss of the Pit.',
         'Astartes Chainsword':'Commorragh, become the boss of the Pit.',
         '[Retobi-Pattern] Plasma Gun':'Commorragh, become the boss of the Pit.',
         'Master Surgeon Gloves':"Upper decks, chapter 3, Marajai's quest.",
         "Arch-Strategist's Visor":"Upper Decks, chapter 3, Marajai's quest.",
         "Saint Cognatius's Scriptures":"""Foulstone, complete project of rank 3,
                                           persuasion check with penalty -40.""",
         "Tempestus Carapace Elite Backpack":"""Chapter 4, Quetza Temer, location with the body of Winterscale's Seneschal, 
                                                tech-use check with penalty -80.""",
        "Bayonet Chain Knife":"Chapter 4, Quetza Temer.",
        "Crucible of Torment":"Chapter 4, Quetza Temer.", 
        "Beggar's Weapon":"Chapter 4, Quetza Temer",
        "Camo Cloak":  "Chapter 4, Quetza Temer",  
        "Suin Kaelean":  "Chapter 4, Quetza Temer",
        "[Craftworld World Origin] Aeldari Guardian Mesh Armour":"Chapter 4, Quetza Temer",
        "Aeldari Farseer Mesh Armour":"Chapter 4, Quetza Temer",
        "Doombringer Hood":"Chapter 4, Quetza Temer, Laboratory, logic check with penalty -75.",
        "Tiara Of Immaterium":"Chapter 4, Quetza Temer, Laboratory.",
        "Archmachinator's Deceit":"Chapter 4, Quetza Temer, Laboratory.",
        "Blood Barrier Emitter":"Chapter 4, Quetza Temer, Laboratory.",
        "Witch Staff":"Chapter 4, Quetza Temer, Laboratory.",
        "Halo Device":"Chapter 4, The system Mundus Nullius, Abandoned palace.",
        "[Fortress World Origin] Tempestus Carapace":"Chapter 4, The system Mundus Nullius, Abandoned palace.",
        "Crusade Plate Gloves":"Chapter 4, The system Mundus Nullius, Abandoned palace.",
        "Bonebreaker Maul":"Chapter 4, The system Mundus Nullius, Abandoned palace.",
        "Archeotech Fusion Gun":"Chapter 4, The system Orsellio Prophecy, allow Marajai to take part in assault.",
        "Plasma Ring":"Chapter 4, The system Smaragdus Mundus, the quest of Cassia.", 
        "Grenadier's Backpack":"Chapter 4, The system Smaragdus Mundus, the quest of Cassia.", 
        "Elite Marksman Gloves":"Chapter 4, The system Smaragdus Mundus, the quest of Cassia.",
        "Lilaethan's Fury":"""Yanus, complete project of rank 5, 
        Yrliet must be in party, check of Awareness with penalty -95,         
        select the line where she is mentioned.""",
        "Swiftshroud Cloak":"Chapter 4, the system Gartrafal's Calamity, the quest of Heinrix.",
        "Instigator's Mantle":"Chapter 4, the system Gartrafal's Calamity, the quest of Heinrix.",
        "Malefic Tome — Belief in the Inscrutable":"Chapter 4, the system Gartrafal's Calamity, the quest of Heinrix.",
        "Tainted Phylactery":"Dargonus, complete project of rank V and success at any skill check",                  

}

In [318]:
output = armors_df_ru.query('Item_Name.str.contains("Панцирь Темпестус")')
display(output)
#show searched row in english table
row_index  = output.index.values
display(armors_df.loc[row_index,'Item_Name'])

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name_Ru
55,44555bd3e9b149408470cefb5da1c9ba,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,9622ca4c7c124db1963f419d42a5b66d,,68287a54721b4dbe9ddc9a2e4ce9e2ee,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",257bc3b704ba49798ae5b1645b3c4bc0,,,,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",d9dd93d556be46d199184a7bdf4407a7,,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,c7a53b43ac5f4f11a1aef612dd3f4f71,590cb5c394ee4b5eb213121c96063dec,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",684c766dc9c24fcf856dc1d6f01d9290,,,,,,,,,,0ba56a61-cf72-40ac-928c-794b4a694855,,d568aaa2-02c0-45e2-bb5e-858a1b30b4a1,,,Armours,Pattern,60.0,4.0,25.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\TempestusCarapaceOrigin1_Item.jbp,[Происхождение: мир-крепость] Панцирь Темпестус,Базовое свойство: +2 к отражению дистанционных атак.\nСвойство мира: +10 к Выносливости.,,,Heavy,,0.0,,,Heavy,Human,,[Происхождение: мир-крепость] Панцирь Темпестус,Эксплораторы_Глава_2,True,,,,,Нет,,Нет,
57,2d262a9870604d3e88260a11c984d725,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,39fef431246e6054c9909c0f9091ba86,9622ca4c7c124db1963f419d42a5b66d,,68287a54721b4dbe9ddc9a2e4ce9e2ee,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",257bc3b704ba49798ae5b1645b3c4bc0,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,c7a53b43ac5f4f11a1aef612dd3f4f71,590cb5c394ee4b5eb213121c96063dec,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,684c766dc9c24fcf856dc1d6f01d9290,,,,,,,,,,,,,,,,318ecd3b-7e7e-4eb5-b266-41ca1f852f12,095003ca-339c-48a5-8283-4398f59cb8f9,,,,Armours,Pattern,50.0,3.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\TempestusCarapace_Item.jbp,Панцирь Темпестус,Базовое свойство: +2 к отражению дистанционных атак.,,,Heavy,,0.0,,,Heavy,Human,,,,False,,,,,Нет,,Нет

55    [Fortress World Origin] Tempestus Carapace
57                            Tempestus Carapace
Name: Item_Name, dtype: object

In [319]:
output = equip_items_df_ru.query('Item_Name.str.contains("Поганая филактерия")')
display(output)
#show searched row in english table
row_index  = output.index.values
display(equip_items_df.loc[row_index,'Item_Name'])

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Item_Name_Ru,Vendor_Name_Ru,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name_Ru
512,68d3bfa288e24641bc4da610e8b7c3a4,"54663362c56fb114c9dc709407dcfc82, BlueprintItemEquipmentRing",,,,,,,f5ac458623114ae42b60f603d82d102a,,029210d48304e8d4f8dee896d96b0527,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",95ac0cecf5734d32a8b273896b24a9c3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,28f01eeb-b7b8-4da7-afc3-f52d108ea38c,13d68192-7682-43f1-aaba-bb7846aaa670,,78765aa4-9889-43fc-80a5-95018a298a73,,Tech,Unique,,,20.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Rings\Unique\TaintedPhylactery_Item.jbp,Поганая филактерия,Персонаж получает +(Знания: Ксеносы / 3)% к шансу критического попадания против ксеносов.,"Таинственный ксенообъект, принадлежавший магусу поганого генокульта с мира-улья Даргонус.",,,,False,,,,,Нет,,Нет,


512    Tainted Phylactery
Name: Item_Name, dtype: object

In [320]:
searched_items = weapons_table_ru[weapons_table_ru['Название'].str.contains('Неистовство')]
row_index  = searched_items.index.values
display(searched_items)
display(weapons_table.loc[row_index,'Item_Name'])


,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет,Item_Location,Planet_Name,Project_Name,Награда_За_Проект,Оружие_За_Проект,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name_Ru
295,1ca7c664e50c4ba39b55b294d5765d84,Неистовство Лилэтан,"Когда персонаж наносит урон с помощью Неистовства Лилэтан, цель оказывается ослеплена до конца боя. После первой за раунд атаки этим оружием против ослепленной цели не требуется период восстановления.",,Примитивное,Нет,Одноручное,Xeno,Unique,Раздирающий,50.0,0.0,26.0,44.0,0.0,1.0,Melee,10.0,0.0,10.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,,,0.0,,,,0.0,,,,0.0,,,,0.0,,,Нет,,,,,Нет,,Нет,


295    Lilaethan's Fury
Name: Item_Name, dtype: object

#### Equipment except armors:

In [321]:
# indexes of the rows with items to add descriptions
items_indexes = equip_items_df_ru[equip_items_df_ru['Item_Name'].isin(items.keys())].index.values.tolist()
print(items_indexes)
# Dataframe with corrected items
changes_items_ru = equip_items_df_ru[['AssetId','Item_Name','Item_Location']].copy(deep = True)
display(changes_items_ru.iloc[items_indexes].head())


[18, 73, 75, 83, 86, 123, 124, 127, 128, 135, 138, 143, 155, 157, 159, 165, 187, 194, 229, 244, 246, 256, 297, 305, 310, 313, 327, 339, 347, 391, 415, 422, 441, 446, 447, 460, 465, 478, 493, 501, 507, 509, 512]


,AssetId,Item_Name,Item_Location
18,9c163228266c4cc892dfb98086d27b52,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,
73,44555bd3e9b149408470cefb5da1c9ba,[Происхождение: мир-крепость] Панцирь Темпестус,
75,2d262a9870604d3e88260a11c984d725,Панцирь Темпестус,
83,9c8d4a592a284b20a9e2fcf4673135e4,Плетеный доспех эльдарского провидца,
86,01fcc1fe911d4ced864f2dd0af7a1e56,Доспех бесстрашия,


weapons_table_ru[weapons_table_ru['Название']=='Игольная винтовка']
equip_items_df_ru.query('Item_Name=="Вуаль обманчивой прозорливости"')[['AssetId','Item_Name','Item_Location','Vendor_Name_Ru']]
t = index_df_ru.query('text.notna()')
t[t['text'].str.contains("Модель Мезоа")]

In [322]:
items_indexes = equip_items_df_ru[equip_items_df_ru['Item_Name'].isin(items.keys())].index.values.tolist()
changes_items_eng = equip_items_df[['AssetId','Item_Name','Item_Location','Vendor_Name']].copy(deep = True)
# Join test_df_items
changes_items = changes_items_eng.join(changes_items_ru[['Item_Name','Item_Location']], lsuffix = '_En', 
                                           rsuffix = '_Ru')

Apply changes to equipment except weapons and armors:

In [323]:
# russian version
changes_items['Item_Location_Ru'] = changes_items['Item_Name_Ru'].map(items)
#english version
changes_items['Item_Location_En'] = changes_items['Item_Name_En'].map(items_eng)
#display(changes_items.iloc[items_indexes][['AssetId','Item_Name_En','Item_Name_Ru','Item_Location_En','Item_Location_Ru']])

Changes in weapons:

In [324]:
# slice of weapons_table_ru for changed items
changes_weapons = weapons_table_ru[weapons_table_ru['Название'].isin(items.keys())].copy(deep = True)
changes_weapons = changes_weapons[['AssetId','Название','Item_Location']]
changes_weapons = changes_weapons.join(weapons_table[['Item_Name','Item_Location']], 
                                       rsuffix='_En',
                                         lsuffix='_Ru')
display(changes_weapons.head())



,AssetId,Название,Item_Location_Ru,Item_Name,Item_Location_En
10,4d820ce4c3134a44bc83bc72af32638c,Цепной меч Астартес,,Astartes Chainsword,
52,5fa48fb54dcc4129927393329c3c4f42,Наперстный огнемет,,Flamer Digi-Weapon,
58,f10b1ba4bc42488bbddbc75515065250,Плазма-кольцо,,Plasma Ring,
79,49d53db2e8eb41ee9e985926fb30c8d6,Фрактальный клинок,Леса Януса,Fractal Blade,Forests of Janus
112,bf521dfa014e473b800ef28bdc8e6ae9,"Пистолет ""Инферно""",,Inferno Pistol,


Apply changes to weapons

In [325]:
# russian version
changes_weapons['Item_Location_Ru'] = changes_weapons['Название'].map(items)
#english version
changes_weapons['Item_Location_En'] = changes_weapons['Item_Name'].map(items_eng)
display(changes_weapons[['AssetId','Item_Name','Название','Item_Location_En','Item_Location_Ru']].head())

,AssetId,Item_Name,Название,Item_Location_En,Item_Location_Ru
10,4d820ce4c3134a44bc83bc72af32638c,Astartes Chainsword,Цепной меч Астартес,"Commorragh, become the boss of the Pit.","Каморра, стать боссом Ямы."
52,5fa48fb54dcc4129927393329c3c4f42,Flamer Digi-Weapon,Наперстный огнемет,"Kiava Gamma Manufactorum, storage, need the password from the ships bridge in the system ""Agathus Archides"".","Мануфакторум Кьявы Гамма, хранилище, нужен код с мостика корабля в системе ""Агатус Архидес""."
58,f10b1ba4bc42488bbddbc75515065250,Plasma Ring,Плазма-кольцо,"Chapter 4, The system Smaragdus Mundus, the quest of Cassia.","Глава 4, система Смарагдус Мундус, квест Кассии."
79,49d53db2e8eb41ee9e985926fb30c8d6,Fractal Blade,Фрактальный клинок,"The system ""Ardematt II"", Recolligers plot.","Система Ардемат II, стоянка реколлигеров."
112,bf521dfa014e473b800ef28bdc8e6ae9,Inferno Pistol,"Пистолет ""Инферно""","The system ""Pillars of Viridice"", Drifting ship of the inquisition.","Система ""Столпы Виридики"", корабль инквизиции."


Changes in armors:

In [326]:
# slice of weapons_table_ru for changed items
changes_armors = armors_df_ru[armors_df_ru['Item_Name'].isin(items.keys())].copy(deep = True)
changes_armors = changes_armors[['AssetId','Item_Name','Item_Location']]
changes_armors = changes_armors.join(armors_df[['Item_Name','Item_Location']], 
                                       rsuffix='_En',
                                         lsuffix='_Ru')
display(changes_armors)

,AssetId,Item_Name_Ru,Item_Location_Ru,Item_Name_En,Item_Location_En
0,9c163228266c4cc892dfb98086d27b52,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,,[Craftworld World Origin] Aeldari Guardian Mesh Armour,
55,44555bd3e9b149408470cefb5da1c9ba,[Происхождение: мир-крепость] Панцирь Темпестус,,[Fortress World Origin] Tempestus Carapace,
57,2d262a9870604d3e88260a11c984d725,Панцирь Темпестус,,Tempestus Carapace,
65,9c8d4a592a284b20a9e2fcf4673135e4,Плетеный доспех эльдарского провидца,,Aeldari Farseer Mesh Armour,
68,01fcc1fe911d4ced864f2dd0af7a1e56,Доспех бесстрашия,,Armour of the Undaunted,


In [327]:
# russian version
changes_armors['Item_Location_Ru'] = changes_armors['Item_Name_Ru'].map(items)
#english version
changes_armors['Item_Location_En'] = changes_armors['Item_Name_En'].map(items_eng)
display(changes_armors[['AssetId','Item_Name_En','Item_Name_Ru','Item_Location_En','Item_Location_Ru']])

,AssetId,Item_Name_En,Item_Name_Ru,Item_Location_En,Item_Location_Ru
0,9c163228266c4cc892dfb98086d27b52,[Craftworld World Origin] Aeldari Guardian Mesh Armour,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Chapter 4, Quetza Temer","Глава 4, Кетца-Темер."
55,44555bd3e9b149408470cefb5da1c9ba,[Fortress World Origin] Tempestus Carapace,[Происхождение: мир-крепость] Панцирь Темпестус,"Chapter 4, The system Mundus Nullius, Abandoned palace.","Глава 4, Система Ничейные миры, заброшенный дворец."
57,2d262a9870604d3e88260a11c984d725,Tempestus Carapace,Панцирь Темпестус,"The system ""Pillars of Viridice"", Drifting ship of the inquisition.","Система ""Столпы Виридики"", корабль инквизиции."
65,9c8d4a592a284b20a9e2fcf4673135e4,Aeldari Farseer Mesh Armour,Плетеный доспех эльдарского провидца,"Chapter 4, Quetza Temer","Глава 4, Кетца-Темер."
68,01fcc1fe911d4ced864f2dd0af7a1e56,Armour of the Undaunted,Доспех бесстрашия,"Planet Grantis, the system ""Forgotten Twins"", if you catch the signal for help.","Планета Грантис, система ""Забытые близнецы"", если успеть на сигнал о помощи."


In [328]:
display(changes_weapons)

,AssetId,Название,Item_Location_Ru,Item_Name,Item_Location_En
10,4d820ce4c3134a44bc83bc72af32638c,Цепной меч Астартес,"Каморра, стать боссом Ямы.",Astartes Chainsword,"Commorragh, become the boss of the Pit."
52,5fa48fb54dcc4129927393329c3c4f42,Наперстный огнемет,"Мануфакторум Кьявы Гамма, хранилище, нужен код с мостика корабля в системе ""Агатус Архидес"".",Flamer Digi-Weapon,"Kiava Gamma Manufactorum, storage, need the password from the ships bridge in the system ""Agathus Archides""."
58,f10b1ba4bc42488bbddbc75515065250,Плазма-кольцо,"Глава 4, система Смарагдус Мундус, квест Кассии.",Plasma Ring,"Chapter 4, The system Smaragdus Mundus, the quest of Cassia."
79,49d53db2e8eb41ee9e985926fb30c8d6,Фрактальный клинок,"Система Ардемат II, стоянка реколлигеров.",Fractal Blade,"The system ""Ardematt II"", Recolligers plot."
112,bf521dfa014e473b800ef28bdc8e6ae9,"Пистолет ""Инферно""","Система ""Столпы Виридики"", корабль инквизиции.",Inferno Pistol,"The system ""Pillars of Viridice"", Drifting ship of the inquisition."
117,6810bc819d164a7c9dd035c8dca82894,Лазбластер,"Каморра, стать боссом Ямы.",Lasblaster,"Commorragh, become the boss of the Pit."
147,447c8afcacb446ad9d19a48527861a27,Игольная винтовка,"Система ""Забытые Близнецы"", мир джунглей, нужно предоставить провизию колонистам. Также в тайном убежище контрабандистов, глава 4.",Needle Rifle,"The system ""Forgotten Twins"", jungle world, need to give food to colonists.Also in the Smuggler Hideout, chapter 4."
158,fad685754a874486acc9278132e169d2,[Модель Мезоа] Плазма-ружье,Мануфакторум Кьявы Гамма.,[Mezoa-Pattern] Plasma Gun,Kiava Gamma Manufactorum.
159,1328a4d8fbc64ae3be79f83028a570d1,[Модель Ретоби] Плазма-ружье,"Каморра, стать боссом Ямы.",[Retobi-Pattern] Plasma Gun,"Commorragh, become the boss of the Pit."
238,29f8b14a7389492299f520115f489f79,Ведьмовской посох,"Глава 4, Кетца-Темер, Лаборатория.",Witch Staff,"Chapter 4, Quetza Temer, Laboratory."


#### Apply changes to base tables

In [329]:
#armors, english
armors_df.loc[armors_df['AssetId'].isin(changes_armors['AssetId']),'Item_Location'
              ] = armors_df['Item_Name'].map(items_eng).fillna(armors_df['Item_Location'])
#armors, russian
armors_df_ru.loc[armors_df_ru['AssetId'].isin(changes_armors['AssetId']),'Item_Location'
                 ] = armors_df_ru['Item_Name'].map(items).fillna(armors_df_ru['Item_Location'])
#weapons, english
weapons_table.loc[weapons_table['AssetId'].isin(changes_weapons['AssetId']),'Item_Location'
                  ] = weapons_table['Item_Name'].map(items_eng).fillna(weapons_table['Item_Location'])
#weapons, russian
weapons_table_ru.loc[weapons_table_ru['AssetId'].isin(changes_weapons['AssetId']),'Item_Location'
                     ] = weapons_table_ru['Название'].map(items).fillna(weapons_table_ru['Item_Location'])
#equipment, english
equip_items_df.loc[
    equip_items_df['AssetId'].isin(changes_items['AssetId']), 'Item_Location'
    ] = equip_items_df['Item_Name'].map(items_eng).fillna(equip_items_df['Item_Location'])
#equipment, russian
equip_items_df_ru.loc[
    equip_items_df_ru['AssetId'].isin(changes_items['AssetId']), 'Item_Location'
    ] = equip_items_df_ru['Item_Name'].map(items).fillna(equip_items_df_ru['Item_Location'])


In [330]:
equip_items_df[equip_items_df['Data_$type'].str.contains("Ring")]['Item_Location'].unique()

array(['', 'Drifting Voidship',
       'Planet: Vheabos VI, project: Holy Servitude',
       "Governor Vyatt's Estate", 'Chasm',
       'Kiava Gamma Manufactorum, storage, need the password from the ships bridge in the system "Agathus Archides".',
       'Anatomical Opera', 'Footfall Shadow Quarters',
       'Kiava Gamma Manufactorum.',
       "Planet: Foulstone, project: In Drusus' Footsteps",
       'Distress Signal Zone', 'Chapter 4, Quetza Temer.',
       'Chapter 4, Quetza Temer, Laboratory.',
       'Unidentified Ruins, the system "Latotians Passage".',
       'Dargonus, construction events',
       'Kiava Gamma, construction events',
       'Chapter 4, The system Mundus Nullius, Abandoned palace.',
       'Janus, construction events',
       'Planet: Kiava Gamma, project: The Motive Force Is Life',
       'Atrium Footfall, sewers. You need to infiltrate the crematorium at Bellardos funeral.',
       "Chapter 4, the system Gartrafal's Calamity, the quest of Heinrix.",
       'Fou

In [331]:
armors_df.loc[armors_df['AssetId'].isin(changes_armors['AssetId'])]

,AssetId,Data_$type,Data_AbilityContainer_Ability1_m_Ability,Data_AbilityContainer_Ability2_m_Ability,Data_AbilityContainer_Ability3_m_Ability,Data_Components_1_AdditionalDescription_Shared_assetguid,Data_Components_1_m_AdditionalString_Shared_assetguid,Data_m_Description_Shared_assetguid,Data_m_DisplayName_Shared_assetguid,Data_m_EquipmentEntity,Data_m_FlavorText_Shared_assetguid,Data_m_Type,Data_Components_0_$type,Data_Components_0_m_Fact,Data_Components_0_m_Facts_0,Data_Components_0_m_Facts_1,Data_Components_0_m_Facts_2,Data_Components_0_m_Property,Data_Components_1_$type,Data_Components_1_m_Fact,Data_Components_1_m_Facts_0,Data_Components_1_m_Facts_1,Data_Components_1_m_Facts_2,Data_Components_1_m_Facts_3,Data_Components_1_m_Property,Data_Components_2_$type,Data_Components_2_m_Fact,Data_Components_2_m_Facts_0,Data_Components_2_m_Facts_1,Data_Components_2_m_Facts_2,Data_Components_2_m_Facts_3,Data_Components_2_m_Property,Data_Components_3_$type,Data_Components_3_m_Facts_0,Data_Components_3_m_Facts_1,Data_Components_3_m_Facts_2,Data_Components_4_$type,Data_Components_4_m_Facts_0,Data_Components_0_AdditionalDescription_m_Key,Data_Components_1_AdditionalDescription_Shared_stringkey,Data_Components_1_AdditionalDescription_m_Key,Data_Components_1_m_AdditionalString_Shared_stringkey,Data_m_Description_Shared_stringkey,Data_m_Description_m_Key,Data_m_DisplayName_Shared_stringkey,Data_m_DisplayName_m_Key,Data_m_FlavorText_Shared_stringkey,Data_m_FlavorText_m_Key,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_CargoVolumePercent,Data_m_Weight,Filepath,Item_Name,Item_Descr,Flavor_Text,Add_Descr,Data_m_Category,Data_m_DamageAbsorbtionPct,Data_m_DamageDeflection_y,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Vendor_Name,Is_Vendor,Item_Location,Planet_Name,Project_Name,Project_Item,Is_Project_Item,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name
0,9c163228266c4cc892dfb98086d27b52,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,35f5651a088d4824951002d4910922fd,,98b410bd49bd4b74b00c78b5b4fba32f,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",3177d0cf74384129877de82675c5355e,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,590cb5c394ee4b5eb213121c96063dec,,,,,,,,,,,,,,,,,,,,,,,70745aa6-22f8-4d76-bb74-868c49fe6b94,,fe562964-fdd5-4da2-9a2b-6071128e8647,,,Armours,Pattern,65.0,0.0,10.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\AeldarGuardianMeshArmour_Item.jbp,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,,Medium,,0.0,,,None,Human,,,False,"Chapter 4, Quetza Temer",,,,False,,False,
55,44555bd3e9b149408470cefb5da1c9ba,"579ddce9e6b6d8e44b05e0715cc66741, BlueprintItemArmor",,,,,,,,9622ca4c7c124db1963f419d42a5b66d,,68287a54721b4dbe9ddc9a2e4ce9e2ee,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",257bc3b704ba49798ae5b1645b3c4bc0,,,,,"65221a9a6133bd0408b019b86642d97e, AddFactToEquipmentWielder",d9dd93d556be46d199184a7bdf4407a7,,,,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",,affa5fdded7e404b910b990f5d344a8c,c7a53b43ac5f4f11a1aef612dd3f4f71,590cb5c394ee4b5eb213121c96063dec,,,"d7b23547716f4a949471625ff6c66fb2, EquipmentRestrictionHasFacts",684c766dc9c24fcf856dc1d6f01d9290,,,,,,,,,,0ba56a61-cf72-40ac-928c-794b4a694855,,d568aaa2-02c0-45e2-bb5e-858a1b30b4a1,,,Armours,Pattern,60.0,4.0,25.0,0.0,e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Armour\PC\TempestusCarapaceOrigin1_Item.jbp,[Fortress World Origin] Tempestus Carapace,Base armour property: +2 deflection against ranged attacks.\nWorld property: +10 Toughness.,,,Heavy,,0.0,,,Heavy,Human,,ExploratorChapter2,True,"Chapter 4, The system Mundus Nullius, Abandoned palace.",,,,False,,False,
57,2d262a9870604d3e88260a11c984d725,"579ddce9e6b6d8e44b05e0715cc66741, Bluepr

## Slicing tables for export 

**Notes**:  
The purpose of this part is to choose what column is go to the save file.

### Equipment tables

#### English

In [332]:
armor_columns = ['Item_Name',
 'Item_Descr',
 'Vendor_Name',
 'Item_Location', 
 'Data_m_Origin',
 'Data_m_Rarity',
 'Data_m_Category',
 'Data_m_DamageAbsorption',
 'Data_m_DamageDeflection',    
 'Data_m_DodgeMod',
 'Data_m_MovePointsAdjustment',
 'Data_m_ProficiencyGroup',
 'Data_m_RaceRestriction',
 'Data_m_RangedHitChanceBonus',
 'Is_Vendor', 'Is_Project_Item',
 'Is_ConstructionEvent_Item']

In [333]:
armors_table =  armors_df[armor_columns]          
armors_table = armors_table.replace('None','')                                 
display(armors_table.head())  

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Data_m_Origin,Data_m_Rarity,Data_m_Category,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
0,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,"Chapter 4, Quetza Temer",Armours,Pattern,Medium,65.0,0.0,,,,Human,,False,False,False
1,Rune Armour,Base armour property: +10% dodge if the wearer's Willpower is more than 50.,,,Xeno,Pattern,Light,30.0,0.0,,,,Human,,False,False,False
2,Improved Armoured Bodyglove,,,,Armours,Common,Light,25.0,0.0,,,,Human,,False,False,False
3,Efficient Armoured Bodyglove,,,Anatomical Opera,Armours,Common,Light,35.0,0.0,,,,Human,,False,False,False
4,Reinforced Armoured Bodyglove,,,,Armours,Common,Light,40.0,0.0,,,,Human,,False,False,False


In [334]:
armors_table[armors_table['Item_Name']=='Armour of Refraction']

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Data_m_Origin,Data_m_Rarity,Data_m_Category,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
94,Armour of Refraction,This carapace grants the wearer +1 deflection and +10% armour if the wearer has moved during their turn.\nBase armour property: +2 deflection against human enemies.,,"Kiava Gamma, construction events",Armours,Unique,Heavy,55.0,3.0,,,Heavy,Human,,False,False,True


In [335]:
select_columns = ['Item_Name',	'Item_Descr','Vendor_Name', 'Item_Location',
                  'Data_m_Origin',	'Data_m_Rarity',	'Data_m_DamageAbsorption',
	              'Data_m_DamageDeflection', 'Is_Vendor', 
	              'Is_Project_Item','Is_ConstructionEvent_Item']

In [336]:
equip_items_df['Data_$type'].unique()
shoulders_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Shoulders")][select_columns]
feet_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Feet")][select_columns]
gloves_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Gloves")][select_columns]
headgear_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Head")][select_columns]
necklaces_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Neck")][select_columns]
rings_table = equip_items_df[equip_items_df['Data_$type'].str.contains("Ring")][select_columns]  

### Russian tables

In [337]:
armor_columns = ['Item_Name', 'Item_Descr',
                 'Vendor_Name_Ru', 'Item_Location', 
                    'Data_m_Origin', 'Data_m_Rarity',
                        'Data_m_Category', 'Data_m_DamageAbsorption',
                        'Data_m_DamageDeflection', 'Data_m_DodgeMod',
                        'Data_m_MovePointsAdjustment', 'Data_m_ProficiencyGroup',
                        'Data_m_RaceRestriction', 'Data_m_RangedHitChanceBonus',
                        'Is_Vendor', 'Is_Project_Item',  'Is_ConstructionEvent_Item']

In [338]:
armors_table_ru =  armors_df_ru[armor_columns]          
armors_table_ru = armors_table_ru.replace('None','')                                 
display(armors_table_ru.head())  

,Item_Name,Item_Descr,Vendor_Name_Ru,Item_Location,Data_m_Origin,Data_m_Rarity,Data_m_Category,Data_m_DamageAbsorption,Data_m_DamageDeflection,Data_m_DodgeMod,Data_m_MovePointsAdjustment,Data_m_ProficiencyGroup,Data_m_RaceRestriction,Data_m_RangedHitChanceBonus,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
0,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Базовое свойство: +15% к броне, если значение Ловкости больше 80.\nСвойство мира: +15 к Ловкости.",,"Глава 4, Кетца-Темер.",Armours,Pattern,Medium,65.0,0.0,,,,Human,,False,Нет,Нет
1,Рунический доспех,"Базовое свойство: +10% к уклонению, если значение Воли больше 50.",,,Xeno,Pattern,Light,30.0,0.0,,,,Human,,False,Нет,Нет
2,Улучшенный бронированный гермокостюм,,,,Armours,Common,Light,25.0,0.0,,,,Human,,False,Нет,Нет
3,Надежный бронированный гермокостюм,,,Анатомическая опера,Armours,Common,Light,35.0,0.0,,,,Human,,False,Нет,Нет
4,Усиленный бронированный гермокостюм,,,,Armours,Common,Light,40.0,0.0,,,,Human,,False,Нет,Нет


In [339]:
select_columns = ['Item_Name',	'Item_Descr',	'Vendor_Name_Ru', 'Item_Location',
                  'Data_m_Origin',	'Data_m_Rarity',	'Data_m_DamageAbsorption',
	'Data_m_DamageDeflection',	'Is_Vendor', 'Is_Project_Item', 'Is_ConstructionEvent_Item']
equip_items_df_ru['Data_$type'].unique()
shoulders_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Shoulders")][select_columns]
feet_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Feet")][select_columns]
gloves_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Gloves")][select_columns]
headgear_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Head")][select_columns]
necklaces_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Neck")][select_columns]
rings_table_ru = equip_items_df_ru[equip_items_df_ru['Data_$type'].str.contains("Ring")][select_columns]  

In [340]:
gloves_table['Item_Location'].unique()

array(['', 'Drifting Voidship', 'Reaving Tempest Spire',
       'Chapter 4, The system Mundus Nullius, Abandoned palace.',
       'Chapter 4, The system Smaragdus Mundus, the quest of Cassia.',
       'Machine Cathedral', 'Upper Decks', 'Ark Mechanicus Hermetico',
       "Upper decks, chapter 3, Marajai's quest.", 'Von Valancius Palace',
       "Governor Vyatt's Estate", 'Forests of Janus',
       'Kiava Gamma Manufactorum', 'Smuggler Hideout',
       'Macro-Cannon Chamber', 'Outer Spire Halls',
       'Planet: Vheabos VI, project: Raider Base',
       'Footfall Shadow Quarters', 'Abandoned Landing Site', 'Chasm'],
      dtype=object)

### Weapons table

#### English version

In [341]:
weapon_table_eng_output_columns = [ 'Item_Name', 
                                   'Item_Descr',
                                   'Vendor_Name',
                                   'Item_Location',
                                     'Family', 'Is_Heavy',
                                       'Holding_Type', 'Origin', 
                                       'Rarity', 'Damage_Type',
                                         'Dodge_Penetration', 'Rate_Of_Fire', 
                                         'Min_Base_Damage', 'Max_Base_Damage',
                                           'Max_Ammo', 'Max_Distance', 'Range',
                                             'Penetration', 'Recoil',                                                
                                               'Damage_Stat_Bonus_Factor', 
                                               'Ability_1_Name', 'Ability_1_Descr', 'Ability1_AP_Cost', 'Ability1_Type', 
                                                'Ability_2_Name', 'Ability_2_Descr', 'Ability2_AP_Cost', 'Ability2_Type',
                                                'Ability_3_Name', 'Ability_3_Descr', 'Ability3_AP_Cost', 'Ability3_Type', 
                                                'Ability_4_Name', 'Ability_4_Descr', 'Ability4_AP_Cost', 'Ability4_Type',
                                                  'Ability_5_Name', 'Ability_5_Descr', 'Ability5_AP_Cost', 'Ability5_Type',
                                                  'Is_Vendor',  'Is_Project_Item',  'Is_ConstructionEvent_Item',
                                                    'Add_Descr']
weapons_table = weapons_table[weapon_table_eng_output_columns]
display(weapons_table.head(1))

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item,Add_Descr
0,Ancient Force Sword,This sword deals an additional +psy rating damage.,,Unidentified Ruins,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,,,0,,,,0,,False,False,False,


#### Russian version

In [342]:
display(weapons_table_ru.head(1))

,AssetId,Название,Описание,Доп_Описание,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,+%_Груза,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракция_Продавец,Фракционный_Предмет,Item_Location,Planet_Name,Project_Name,Награда_За_Проект,Оружие_За_Проект,Dialog_Item_Location,Is_ConstructionEvent_Item,Area_Name_Ru
0,24db44debaed469386fad90088c20ee9,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,15.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,,Нет,Неизвестные руины,,,,Нет,,Нет,


In [343]:
weapons_table_ru_output_columns = [ 'Название', 
                                   'Описание',
                                   'Фракция_Продавец',
                                   'Item_Location',
                                     'Семейство', 'Тяжёлое',
                                    'Двуручное_или_Одноручное', 'Категория', 'Редкость',
                                    'Тип_Урона', 'Пробитие_Уклонения', 'Скорострельность',
                                    'Мин_Базовый_Урон', 'Макс_Базовый_Урон', 'Боезапас', 'Макс_Дальность',
                                    'Ближнее_или_Дистанц', 'Пробитие_Брони', 'Отдача', 
       'Фактор_Урона', 'Название_Способности_1', 'Описание_Способности_1',
       'Стоимость_ОД_Способности_1', 'Тип_Действия_Способности_1',
       'Название_Способности_2', 'Описание_Способности_2',
       'Стоимость_ОД_Способности_2', 'Тип_Действия_Способности_2',
       'Название_Способности_3', 'Описание_Способности_3',
       'Стоимость_ОД_Способности_3', 'Тип_Действия_Способности_3',
       'Название_Способности_4', 'Описание_Способности_4',
       'Стоимость_ОД_Способности_4', 'Тип_Действия_Способности_4',
       'Название_Способности_5', 'Описание_Способности_5',
       'Стоимость_ОД_Способности_5', 'Тип_Действия_Способности_5',
        'Фракционный_Предмет',  'Оружие_За_Проект', 'Is_ConstructionEvent_Item',
       'Доп_Описание']
weapons_table_ru = weapons_table_ru[weapons_table_ru_output_columns]

In [344]:
print('The number of columns in eng and ru weapon table: ',len(weapons_table.columns),', ', len(weapons_table_ru.columns))

The number of columns in eng and ru weapon table:  44 ,  44


________

## Format Tables

**Notes:**  
The purpose of this part is to polish final look of the table like rename columns or change some values.

### English

In [345]:
dfs = [armors_table, shoulders_table, feet_table, gloves_table, headgear_table, necklaces_table, rings_table]
#drop empty columns
for df in dfs:
    for col in df:
        if(len(df.loc[:,col].unique())==1):
            df.pop(col)
    df.columns = map(column_name_formatter, df.columns)

In [346]:
for df in dfs:
    display(df.head())

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Category,DamageAbsorption,DamageDeflection,ProficiencyGroup,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
0,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,"Chapter 4, Quetza Temer",Armours,Pattern,Medium,65.0,0.0,,False,False,False
1,Rune Armour,Base armour property: +10% dodge if the wearer's Willpower is more than 50.,,,Xeno,Pattern,Light,30.0,0.0,,False,False,False
2,Improved Armoured Bodyglove,,,,Armours,Common,Light,25.0,0.0,,False,False,False
3,Efficient Armoured Bodyglove,,,Anatomical Opera,Armours,Common,Light,35.0,0.0,,False,False,False
4,Reinforced Armoured Bodyglove,,,,Armours,Common,Light,40.0,0.0,,False,False,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor
121,Arch-Militant Hood,Each of the wearer's critical hits made with a ranged weapon increases their melee weapon critical damage by +STR bonus until the end of combat.,,,Textile,Common,False
122,Shroud of Dauntlessness,"Whenever the wielder performs a heroic act or a desperate measure, they gain +2 AP, +6 MP, and (2 × TGH bonus) temporary wounds.",,Ruined Palace,Tech,Common,False
123,Cameleoline Cloak,Mimic fibres blend the colouration of the wearer into their surroundings. The cloak grants the wearer +10% dodge.,,"The system ""Ardematt II"", Recolligers plot.",Xeno,Common,False
124,Camo Cloak,"While the wearer is adjacent to full cover, their dodge chance is calculated without considering enemies' Perception value.",,"Chapter 4, Quetza Temer",Textile,Common,False
125,Commissar's Cloak,The wearer gains the Get Back In the Fight! ability that costs 0 AP.,,,Textile,Common,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor,Is_Project_Item
177,Battle-Psyker Boots,The wearer gains PER bonus% dodge until the start of their next turn for each enemy affected by their psy-power or psyker staff attack. Stacks.,,Macro-Cannon Chamber,Textile,Common,False,False
178,Boots of Decay,"Each time the wearer uses a major psychic power, all enemies in their line of sight lose 30% of their deflection until the start of the wearer's next turn.",,,Tech,Common,False,False
179,Boots of Kasrkin,"When in cover, the wearer has 100% cover efficiency against area attacks but never dodges them. Also, they suffer −50% less damage from area attacks.\nWhenever the wearer uses a grenade and it hits at least 3 enemies at once for the first time in combat, this action does not spend AP.",DrusianChapter4,,Tech,Common,True,False
180,Breacher Boots,"Whenever the wearer's area attack damages at least two enemies within a 2-cell radius from the wearer, the wearer gains the Kick ability costing 0 AP to use until the end of their turn.",PirateChapter2,,Textile,Common,True,False
181,Breezecatcher Boots,"Whenever the wearer moves through a cell occupied by an enemy, the wearer gains +15% dodge against that enemy's attacks until the wearer's next turn.",,Von Valancius Palace,Textile,Common,False,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor,Is_Project_Item
235,Abyssal Conduit,Every second use of a psyker staff's unique psychic power deals +(2 × WP bonus) additional damage.,KasballicaMissionChapter4,,Tech,Common,True,False
236,Abyss Weaver Gloves,"Allies targeted by the wearer's abilities gain +20% of the wearer's armour penetration and +30% of the wearer's critical damage for 1 round.\nIf that ally scores a critical hit by the start of the wearer's next turn, the wearer's next attack is always a critical hit.",,,Miscellaneous,Common,False,False
237,Adaptive Gloves,Increases the wearer's Weapon Skill by 10 and parry by 10%.,,,Tech,Common,False,False
238,Augmented Gloves,Grants the wearer +10 Weapon Skill and +5 Strength.,,Drifting Voidship,Tech,Common,False,False
239,Avenger Gauntlets,All attacks of opportunity deal an additional +AGI bonus damage.,,,Tech,Common,False,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor,Is_Project_Item
291,Poet's Grace,The wearer gains +25 Commerce and +25 Persuasion.,PirateChapter2,,Tech,Common,True,False
292,Crusade Plate Helmet,The wearer's heavy weapon attacks gain +50% additional damage but never critically hit.,DrusianChapter4,,Tech,Common,True,False
293,Cultist Hood,"Requires Heretical — Adherent\nWhen the wearer uses a psychic power attack, they either restore 2 AP or take 10 warp damage with a 50% chance for each of those effects.",,,Miscellaneous,Common,False,False
294,Deck Officer's Helmet,Grants +5% armour.,,Upper Decks,Tech,Unique,False,False
295,Virtue of Contempt,"Each time the wearer inflicts burning from flamers, that burning level is doubled.",DrusianChapter4,,Tech,Common,True,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
351,Conversion Field,"Whenever the wearer suffers a critical hit for the first time in a round, all damage from this hit is prevented.\nAfter that, a blinded negative effect is inflicted on all characters in a 2-cell radius around the wearer until the end of the wearer's next turn.",,,Xeno,Common,False,False,False
352,Daemon Tool,"Whenever the wearer deals warp damage, the damaged character also suffers burning (3). Burning effects inflicted by the wearer have their power increased by +veil degradation.",,,Miscellaneous,Common,False,False,False
353,Aquila Pendant,"Grants +10 Willpower.\nIf the wearer has Conviction — Dogmatic, they also gain +5 Fellowship.",,,Textile,Common,False,False,False
354,Hope Pendant,"Requires Iconoclast — Votary\nAt the start of each turn (except for extra turns), the wearer gains +resolve momentum for each ally (including themselves) with 50% or less wounds.",,,Tech,Common,False,False,False
355,Invigorating Resolve,"Whenever the wearer and allies in a 2-cells radius gain momentum, they also gain an equal amount of temporary wounds.",ExploratorChapter2,,Miscellaneous,Common,True,False,False


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
406,Accelerator,"Whenever the wearer starts their turn with no enemies adjacent, they gain +6 MP.",,,Miscellaneous,Common,False,False,False
407,Auspex,Grants a +10 bonus to Awareness.,,,Tech,Common,False,False,False
408,Blades Clone Field,"Whenever the wearer fails to parry an attack, they can reroll the parry test once.\nIf they wield dual melee weapons, they can reroll the parry test twice.",PirateChapter4,,Miscellaneous,Common,True,False,False
409,Combi-Tool,Grants a +10 bonus to Tech-Use.,,,Tech,Common,False,False,False
410,Diagnostor,Grants +15 Medicae.,,Drifting Voidship,Tech,Common,False,False,False


#### Weapons table

In [347]:
weapons_copy = weapons_table.copy(deep = True)

Format column names:

In [348]:
#Create dict with column names as keys and output column names as values
weapon_table_eng_transcription = dict(zip(weapons_table.columns.tolist(), weapons_table.columns.tolist()))
#rename columns in accordance with the created dict
weapons_table = weapons_table.rename(columns=weapon_table_eng_transcription)

Replace nulls in 'Ability_AP_Costs' with empty row if all other fields of that ability number is empty:

In [349]:
for i in range(1,6):
    weapons_table.loc[(weapons_table['Ability_{}_Name'.format(i)]=='') 
                 & (weapons_table['Ability_{}_Descr'.format(i)]=='') 
                 & (weapons_table['Ability{}_Type'.format(i)]==''),'Ability{}_AP_Cost'.format(i)] = ''

Replacing values:

In [350]:
weapons_table['Family'].unique()
weapons_table['Family'] = weapons_table['Family'].replace({'Solid':'Solid projectile'})

Check for duplicates:

In [351]:
weapons_table.duplicated().sum()
weapons_table[weapons_table.duplicated()]

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item,Add_Descr
92,Heavy Bolter,,DrusianChapter2,,Bolt,True,TwoHanded,RangedWeaponry,Common,Impact,0,8,6,9,96,24,Ranged,20,40,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Burst Fire,Burst fire with equipped ranged weapon.,3,Burst,Reload,Reloads equipped weapon.,4,Reload,,,,,,,,,True,False,False,
136,Bolter,,,,Bolt,False,TwoHanded,RangedWeaponry,Common,Impact,0,3,9,14,14,24,Ranged,15,25,One,Single Shot,Single shot with equipped ranged weapon.,1,SingleShot,Burst Fire,Burst fire with equipped ranged weapon.,2,Burst,Reload,Reloads equipped weapon.,3,Reload,,,,,,,,,False,False,False,
324,Uralon's Accursed Crozius,"On a successful hit with this crozius, all of the wielder's allies gain +5 to damage against the target for 1 round.\nAfter the wielder deals damage with this crozius, each of the wielder's allies performs an attack of opportunity against the target.",,,Power,False,,Tech,Unique,Energy,0,1,40,50,0,1,,35,0,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,,,,,,,,,,,,,,,,,False,False,False,


Drop duplicates

In [352]:
weapons_table = weapons_table.drop_duplicates().reset_index(drop = True)

In [353]:
#Look of the table after formatting
weapons_table.head(2)

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item,Add_Descr
0,Ancient Force Sword,This sword deals an additional +psy rating damage.,,Unidentified Ruins,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,,,,,,,,,False,False,False,
1,[Mezoa-Pattern] Arc Rifle,"When you hit the target with an arc rifle, you also attack any enemies within 2 cells of the target.",ExploratorChapter2,,Solid projectile,False,TwoHanded,RangedWeaponry,Pattern,Shock,0,1,13,20,4,18,Ranged,30,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,,,,,,,,,,,,,True,False,False,


### Table formatting Ru

#### Armor table

In [354]:
armor_columns_transcription = {'Item_Name':'Название',
                 'Item_Descr':'Описание', 
                 'Data_m_Origin':'Категория',
                 'Data_m_Rarity':'Редкость',
                 'Data_m_Category':'Тип',
                 'Data_m_DamageAbsorption':'Поглощение_Урона',
                 'Data_m_DamageDeflection':'Отражение_Урона',                                   
                 'Data_m_ProficiencyGroup':'Категория_Брони',
                 'Vendor_Name_Ru':'Фракция_Продавец',
                 'Is_Vendor':'Фракционный_Предмет',
                 'Item_Location':'Локация',
                 'Is_Project_Item':'Броня_За_Проект',
                 'Is_ConstructionEvent_Item':'Предмет_За_Событие_Постройки'
                 }

In [355]:
for col in armors_table_ru:
        if(len(armors_table_ru.loc[:,col].unique())==1):
            armors_table_ru.pop(col)                                 
armors_table_ru = armors_table_ru.rename(columns = armor_columns_transcription)
armors_table_ru = armors_table_ru.replace('None','')
display(armors_table_ru.head())  

,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Тип,Поглощение_Урона,Отражение_Урона,Категория_Брони,Фракционный_Предмет,Броня_За_Проект,Предмет_За_Событие_Постройки
0,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Базовое свойство: +15% к броне, если значение Ловкости больше 80.\nСвойство мира: +15 к Ловкости.",,"Глава 4, Кетца-Темер.",Armours,Pattern,Medium,65.0,0.0,,False,Нет,Нет
1,Рунический доспех,"Базовое свойство: +10% к уклонению, если значение Воли больше 50.",,,Xeno,Pattern,Light,30.0,0.0,,False,Нет,Нет
2,Улучшенный бронированный гермокостюм,,,,Armours,Common,Light,25.0,0.0,,False,Нет,Нет
3,Надежный бронированный гермокостюм,,,Анатомическая опера,Armours,Common,Light,35.0,0.0,,False,Нет,Нет
4,Усиленный бронированный гермокостюм,,,,Armours,Common,Light,40.0,0.0,,False,Нет,Нет


In [356]:
armors_table_ru.query('Броня_За_Проект=="Да"')

,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Тип,Поглощение_Урона,Отражение_Урона,Категория_Брони,Фракционный_Предмет,Броня_За_Проект,Предмет_За_Событие_Постройки
67,Броня Арбитратора Соломона Вира,"Если противник, получивший попадание, является преступником, он получает +1 уровень эффекта замешательство на 2 раунда.\nБазовое свойство: +(Напор / 10) к отражению.",,"Планета: Даргонус, проект: Адептус Арбайтес",Armours,Unique,Heavy,85.0,5.0,Heavy,False,Да,Нет
101,Одеяние верного,"Если персонаж не двигается в свой ход, эффективность укрытия для него повышается на 20% на 1 раунд. Если персонаж в свой ход перемещается хотя бы на 3 клетки, его уклонение увеличивается на 20% на 1 раунд.\nБазовое свойство: +20% к уклонению от дистанционных атак.",,"Планета: Даргонус, проект: Скола Прогениум",Armours,Unique,Light,50.0,0.0,,False,Да,Нет


In [357]:
armors_table_ru.query('Предмет_За_Событие_Постройки=="Да"')

,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Тип,Поглощение_Урона,Отражение_Урона,Категория_Брони,Фракционный_Предмет,Броня_За_Проект,Предмет_За_Событие_Постройки
94,Панцирь рефракции,"Этот панцирь увеличивает отражение на 1 и броню на 10%, если персонаж двигается в свой ход.\nБазовое свойство: +2 к отражению атак врагов-людей.",,"Кьява Гамма, строительство проектов",Armours,Unique,Heavy,55.0,3.0,Heavy,False,Нет,Да


In [358]:
for col in armors_table_ru.columns:
    print(armors_table_ru[col].unique())

['[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца'
 'Рунический доспех' 'Улучшенный бронированный гермокостюм'
 'Надежный бронированный гермокостюм'
 'Усиленный бронированный гермокостюм'
 '[Происхождение: мир смерти] Бронированный гермокостюм'
 'Бронированный гермокостюм' 'Панцирный нагрудник' 'Каркасный нагрудник'
 'Керамитовый штурмовой панцирь' 'Кольчуга'
 '[Происхождение: демонический мир] Панцирь бойца Хаоса'
 '[Происхождение: мир войны] Доспехи крестоносца' 'Доспех Арлекина'
 'Доспех эльдарского странника' '[Происхождение: мир войны] Панцирь бойца'
 'Панцирь бойца' 'Улучшенный легкий панцирь бойца'
 '[Происхождение: мир смерти] Легкий панцирь бойца' 'Легкий панцирь бойца'
 'Фамильная силовая броня'
 '[Происхождение: мир войны] Фехтовальный гермокостюм'
 'Усиленный флак-жилет' 'Улучшенный флак-жилет' 'Флак-жилет'
 'Доспех с мира-кузни' 'Броня из призрачных пластин'
 'Плетеный доспех гладиатора' 'Улучшенная противоосколочная броня'
 'Противоосколочная броня' 'Уси

### Other tables

In [359]:
rings_table_ru.head(1)

,Item_Name,Item_Descr,Vendor_Name_Ru,Item_Location,Data_m_Origin,Data_m_Rarity,Data_m_DamageAbsorption,Data_m_DamageDeflection,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
406,Ускоритель,"Когда в начале хода персонажа на соседних клетках нет врагов, персонаж получает +6 ОП.",,,Miscellaneous,Common,,,False,Нет,Нет


In [360]:
dfs_columns_transcription = {   'Item_Name':'Название',
                                'Item_Descr':'Описание',
                                'Add_Descr':'Доп_Описание',
                                'Data_m_Origin':'Категория',
                                'Data_m_Rarity':'Редкость',
                                'Data_m_DamageAbsorption':'Поглощение_Урона',
                                'Data_m_DamageDeflection':'Отражение_Урона',
                                'Data_m_CargoVolumePercent':'+%_Груза',                                
                                'Vendor_Name_Ru':'Фракция_Продавец',
                                'Is_Vendor':'Фракционный_Предмет',
                                'Item_Location':'Локация',
                                'Is_Project_Item':'Предмет_За_Проект',
                                'Is_ConstructionEvent_Item':'Предмет_За_Событие_Постройки'}    

In [361]:
rings_table_ru = rings_table_ru.rename(columns = dfs_columns_transcription)
shoulders_table_ru = shoulders_table_ru.rename(columns = dfs_columns_transcription)
feet_table_ru = feet_table_ru.rename(columns = dfs_columns_transcription)
gloves_table_ru = gloves_table_ru.rename(columns = dfs_columns_transcription)
headgear_table_ru = headgear_table_ru.rename(columns = dfs_columns_transcription)
necklaces_table_ru = necklaces_table_ru.rename(columns = dfs_columns_transcription)



In [362]:
dfs = [shoulders_table_ru, feet_table_ru, gloves_table_ru, headgear_table_ru, necklaces_table_ru, rings_table_ru]
#drop empty columns
for df in dfs:    
    for col in df:
        if(len(df.loc[:,col].unique())==1):
            df.pop(col)   

In [363]:
for df in dfs:
    display(df.head())

,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет
121,Накидка архмилитанта,"Каждое критическое попадание, которое персонаж совершает дистанционным оружием, увеличивает критический урон, который он наносит рукопашным оружием, на бонус СИЛ до конца боя.",,,Textile,Common,False
122,Покров неустрашимости,"Совершая героический акт или прибегая к отчаянным мерам, персонаж получает +2 ОД, +6 ОП и (2 × бонус ВЫН) временного здоровья.",,Разрушенный дворец,Tech,Common,False
123,Хамелеолиновый плащ,Благодаря мимикрирующему волокну облачение персонажа копирует цвет окружения. Плащ дает +10% к уклонению.,,"Система Ардемат II, стоянка реколлигеров.",Xeno,Common,False
124,Маскирующий плащ,"Если персонаж находится за полным укрытием, шанс его уклонения рассчитывается без учета Восприятия противника.",,"Глава 4, Кетца-Темер.",Textile,Common,False
125,Плащ комиссара,"Персонаж получает способность Никаких оправданий!, которая не требует затрат ОД.",,,Textile,Common,False


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет,Предмет_За_Проект
177,Боевые сапоги псайкера,"Персонаж получает бонус ВОС% к уклонению до начала своего следующего хода за каждого противника, к которому применена его психосила или атака посоха. Эффект суммируется.",,Палата макропушки,Textile,Common,False,Нет
178,Сапоги разложения,"Когда персонаж применяет значимую психосилу, все противники в его зоне видимости теряют 30% от своего отражения до начала следующего хода персонажа.",,,Tech,Common,False,Нет
179,Сапоги касркина,При нахождении за укрытием его эффективность для персонажа при атаках по области становится равна 100%. При этом персонаж не уклоняется от атак по области и получает от них на 50% меньше урона.\nКогда персонаж использует гранату и впервые за бой как минимум 3 противника одновременно получают от ...,Друзианцы_Глава_4,,Tech,Common,True,Нет
180,Пробивные сапоги,"Когда персонаж совершает атаку по области, нанося урон хотя бы двум игрокам в радиусе 2 клеток от себя, он получает способность Удар ногой, не требующую затрат ОД, которую можно применить до конца хода.",ПустотноеБратство_Глава_2,,Textile,Common,True,Нет
181,Сапоги попутного ветра,"Перемещаясь через клетку, занятую противником, персонаж получает +15% к уклонению от атак этого противника до своего следующего хода.",,Дворец фон Валанциус,Textile,Common,False,Нет


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет,Предмет_За_Проект
235,Проводник бездны,"Каждая вторая атака, совершенная с помощью посоха псайкера, в каждом ходу наносит +(2 × бонус ВОЛ)% урона дополнительно.",МиссияКасбаллики_Глава_4,,Tech,Common,True,Нет
236,Перчатки ткача бездны,"Союзники, ставшие целями способностей персонажа, получают 20% его пробития брони и 30% его критического урона на 1 раунд.\nЕсли такой союзник совершает критическое попадание до начала следующего хода персонажа, то попадание следующей атаки персонажа всегда считается критическим.",,,Miscellaneous,Common,False,Нет
237,Адаптивные перчатки,Дают +10 к Ближнему Бою +10% к парированию.,,,Tech,Common,False,Нет
238,Аугментированные перчатки,Дают +10 к Ближнему Бою и +5 к Силе.,,Дрейфующий корабль,Tech,Common,False,Нет
239,Наручи мстителя,Все атаки по возможности наносят на бонус ЛВК урона больше.,,,Tech,Common,False,Нет


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет,Предмет_За_Проект
291,Грация стихотворца,Дает +25 к Коммерции и +25 к Убеждению.,ПустотноеБратство_Глава_2,,Tech,Common,True,Нет
292,Шлем крестового похода,"Атаки тяжелым оружием наносят на 50% больше урона, при этом критических попаданий не происходит.",Друзианцы_Глава_4,,Tech,Common,True,Нет
293,Капюшон культиста,Требуется: Еретик — сторонник\nПри применении психосилы персонаж с равной вероятностью может восстановить 2 ОД или получить 10 варп-урона.,,,Miscellaneous,Common,False,Нет
294,Шлем дежурного офицера,Этот шлем дает +5% к броне.,,Верхние палубы,Tech,Unique,False,Нет
295,Добродетель презрения,"Каждый раз, когда персонаж накладывает горение с помощью огнемета, уровень горения удваивается.",Друзианцы_Глава_4,,Tech,Common,True,Нет


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет,Предмет_За_Проект,Предмет_За_Событие_Постройки
351,Преобразовательное поле,При получении первого критического попадания за раунд персонаж игнорирует весь урон от этого попадания.\nПосле этого на все цели в радиусе 2 клеток вокруг персонажа накладывается негативный эффект ослепления до конца следующего хода персонажа.,,,Xeno,Common,False,Нет,Нет
352,Демонический инструмент,"Когда персонаж наносит варп-урон, на цель также накладывается горение (3). Накладываемые персонажем эффекты горения усиливаются на +истончение завесы.",,,Miscellaneous,Common,False,Нет,Нет
353,Подвеска с аквилой,"Дает +10 к Воле.\nЕсли персонаж обладает убеждением Догматик, он также получает +5 к Общительности.",,,Textile,Common,False,Нет,Нет
354,Кулон надежды,"Требуется: Схизматик — поборник\nКаждый раз в начале хода (за исключением дополнительных ходов) персонаж получает решимость моментума за каждого союзника, у которого осталось меньше 50% здоровья, а также за себя при соответствии этому условию.",,,Tech,Common,False,Нет,Нет
355,Бодрящая решимость,"Когда персонаж или его союзники в радиусе 2 клеток получают моментум, они также получают такое же количество временного здоровья.",Эксплораторы_Глава_2,,Miscellaneous,Common,True,Нет,Нет


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Фракционный_Предмет,Предмет_За_Проект,Предмет_За_Событие_Постройки
406,Ускоритель,"Когда в начале хода персонажа на соседних клетках нет врагов, персонаж получает +6 ОП.",,,Miscellaneous,Common,False,Нет,Нет
407,Ауспекс,Дает +10 к Наблюдательности.,,,Tech,Common,False,Нет,Нет
408,Поле удвоения клинков,"Когда персонажу не удается парировать атаку, он может один раз повторить попытку парирования.\nЕсли он использует два рукопашных оружия, то может повторять попытку до двух раз.",ПустотноеБратство_Глава_4,,Miscellaneous,Common,True,Нет,Нет
409,Комби-тул,Дает +10 к Технике.,,,Tech,Common,False,Нет,Нет
410,Диагностор,Дает +15 к Медицине.,,Дрейфующий корабль,Tech,Common,False,Нет,Нет


#### Weapons table

In [364]:
# The table before formatting
display(weapons_table_ru.head(2))

,Название,Описание,Фракция_Продавец,Item_Location,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракционный_Предмет,Оружие_За_Проект,Is_ConstructionEvent_Item,Доп_Описание
0,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Неизвестные руины,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,0.0,,,,0.0,,Нет,Нет,Нет,
1,[Модель Мезоа] Электродуговое ружье,При попадании в цель из электродугового ружья вы также атакуете всех противников в радиусе 2 клеток от цели.,Эксплораторы_Глава_2,,Сплошное,Нет,Двуручное,RangedWeaponry,Pattern,Электрический,0.0,1.0,13.0,20.0,4.0,18.0,Ranged,30.0,10.0,One,Одиночный выстрел,Одиночный выстрел из используемого дистанционного оружия.,2.0,SingleShot,Перезарядка,Перезаряжает выбранное оружие.,3.0,Reload,,,0.0,,,,0.0,,,,0.0,,Да,Нет,Нет,


In [365]:
weapons_copy_ru = weapons_table_ru.copy(deep=True)

Renaming columns:

In [366]:
#Create dict with column names as keys and output column names as values
weapon_table_ru_column_transcription = dict(zip(weapons_table_ru.columns.tolist(),weapons_table_ru.columns.tolist() ))
# Rename columns
weapon_table_ru_column_transcription['Item_Location']='Локация'
weapon_table_ru_column_transcription['Is_ConstructionEvent_Item']='Оружие_За_Событие_Постройки'
#Apply changes to the table
weapons_table_ru = weapons_table_ru.rename(columns = weapon_table_ru_column_transcription)



Replace nulls in weapons abilitites ap cost columns with an empty string if all other ability fields are wmpty:

In [367]:
for i in range(1,6):
    weapons_table_ru.loc[(weapons_table_ru['Название_Способности_{}'.format(i)]=='') 
                 & (weapons_table_ru['Описание_Способности_{}'.format(i)]=='') 
                 & (weapons_table_ru['Тип_Действия_Способности_{}'.format(i)]==''),'Стоимость_ОД_Способности_{}'.format(i)] = ''

Replacing values:


In [368]:
weapons_table_ru['Семейство'].unique()
weapons_table_ru['Семейство'] = weapons_table_ru['Семейство'].replace({'Сплошное':'Пулевое'})

In [369]:
weapons_table_ru = weapons_table_ru.drop_duplicates().reset_index(drop = True)

In [370]:
#look after format is done
display(weapons_table_ru.head(1))

,Название,Описание,Фракция_Продавец,Локация,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракционный_Предмет,Оружие_За_Проект,Оружие_За_Событие_Постройки,Доп_Описание
0,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Неизвестные руины,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,,,,,,,Нет,Нет,Нет,


## Final results

In [371]:
display(weapons_table.head(2))

,Item_Name,Item_Descr,Vendor_Name,Item_Location,Family,Is_Heavy,Holding_Type,Origin,Rarity,Damage_Type,Dodge_Penetration,Rate_Of_Fire,Min_Base_Damage,Max_Base_Damage,Max_Ammo,Max_Distance,Range,Penetration,Recoil,Damage_Stat_Bonus_Factor,Ability_1_Name,Ability_1_Descr,Ability1_AP_Cost,Ability1_Type,Ability_2_Name,Ability_2_Descr,Ability2_AP_Cost,Ability2_Type,Ability_3_Name,Ability_3_Descr,Ability3_AP_Cost,Ability3_Type,Ability_4_Name,Ability_4_Descr,Ability4_AP_Cost,Ability4_Type,Ability_5_Name,Ability_5_Descr,Ability5_AP_Cost,Ability5_Type,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item,Add_Descr
0,Ancient Force Sword,This sword deals an additional +psy rating damage.,,Unidentified Ruins,Force,False,OneHanded,MeleeWeaponry,Pattern,Energy,0,0,14,20,0,1,Melee,30,0,One,Strike,Single attack with equipped melee weapon.,1,SingleShot,Cleave,Wide slashing strike that affects multiple enemies at once.,2,AOE,Psychic Force,"Special attack that counts as a major psychic power.\nIt deals an additional (2 × psy rating + WP bonus) damage, but may trigger psychic phenomena and perils of the warp.",2,Custom,,,,,,,,,False,False,False,
1,[Mezoa-Pattern] Arc Rifle,"When you hit the target with an arc rifle, you also attack any enemies within 2 cells of the target.",ExploratorChapter2,,Solid projectile,False,TwoHanded,RangedWeaponry,Pattern,Shock,0,1,13,20,4,18,Ranged,30,10,One,Single Shot,Single shot with equipped ranged weapon.,2,SingleShot,Reload,Reloads equipped weapon.,3,Reload,,,,,,,,,,,,,True,False,False,


In [372]:
display(weapons_table_ru.head(2))

,Название,Описание,Фракция_Продавец,Локация,Семейство,Тяжёлое,Двуручное_или_Одноручное,Категория,Редкость,Тип_Урона,Пробитие_Уклонения,Скорострельность,Мин_Базовый_Урон,Макс_Базовый_Урон,Боезапас,Макс_Дальность,Ближнее_или_Дистанц,Пробитие_Брони,Отдача,Фактор_Урона,Название_Способности_1,Описание_Способности_1,Стоимость_ОД_Способности_1,Тип_Действия_Способности_1,Название_Способности_2,Описание_Способности_2,Стоимость_ОД_Способности_2,Тип_Действия_Способности_2,Название_Способности_3,Описание_Способности_3,Стоимость_ОД_Способности_3,Тип_Действия_Способности_3,Название_Способности_4,Описание_Способности_4,Стоимость_ОД_Способности_4,Тип_Действия_Способности_4,Название_Способности_5,Описание_Способности_5,Стоимость_ОД_Способности_5,Тип_Действия_Способности_5,Фракционный_Предмет,Оружие_За_Проект,Оружие_За_Событие_Постройки,Доп_Описание
0,Древний психосиловой меч,Этот меч наносит +пси-рейтинг урона дополнительно.,,Неизвестные руины,Силовое,Нет,Одноручное,MeleeWeaponry,Pattern,Энергетический,0.0,0.0,14.0,20.0,0.0,1.0,Melee,30.0,0.0,One,Удар,Одиночная атака с помощью используемого рукопашного оружия.,1.0,SingleShot,Режущий удар,"Сильный режущий удар, который поражает сразу нескольких противников.",2.0,AOE,Пси-усиление,"Особая атака, которая считается значимой психосилой.\nНаносит (2 × пси-рейтинг + бонус ВОЛ) урона дополнительно, но может вызвать пси-морок и угрозы варпа.",2.0,Custom,,,,,,,,,Нет,Нет,Нет,
1,[Модель Мезоа] Электродуговое ружье,При попадании в цель из электродугового ружья вы также атакуете всех противников в радиусе 2 клеток от цели.,Эксплораторы_Глава_2,,Пулевое,Нет,Двуручное,RangedWeaponry,Pattern,Электрический,0.0,1.0,13.0,20.0,4.0,18.0,Ranged,30.0,10.0,One,Одиночный выстрел,Одиночный выстрел из используемого дистанционного оружия.,2.0,SingleShot,Перезарядка,Перезаряжает выбранное оружие.,3.0,Reload,,,,,,,,,,,,,Да,Нет,Нет,


In [373]:
print('Armors, English'.center(30))
display(armors_table.head(2))
print('Armors, Russian'.center(30))
display(armors_table_ru.head(2))

       Armors, English        


,Item_Name,Item_Descr,Vendor_Name,Item_Location,Origin,Rarity,Category,DamageAbsorption,DamageDeflection,ProficiencyGroup,Is_Vendor,Is_Project_Item,Is_ConstructionEvent_Item
0,[Craftworld World Origin] Aeldari Guardian Mesh Armour,Base armour property: +15% armour if the wearer's Agility is more than 80.\nWorld property: +15 Agility.,,"Chapter 4, Quetza Temer",Armours,Pattern,Medium,65.0,0.0,,False,False,False
1,Rune Armour,Base armour property: +10% dodge if the wearer's Willpower is more than 50.,,,Xeno,Pattern,Light,30.0,0.0,,False,False,False


       Armors, Russian        


,Название,Описание,Фракция_Продавец,Локация,Категория,Редкость,Тип,Поглощение_Урона,Отражение_Урона,Категория_Брони,Фракционный_Предмет,Броня_За_Проект,Предмет_За_Событие_Постройки
0,[Происхождение: мир-корабль] Плетеный доспех эльдарского провидца,"Базовое свойство: +15% к броне, если значение Ловкости больше 80.\nСвойство мира: +15 к Ловкости.",,"Глава 4, Кетца-Темер.",Armours,Pattern,Medium,65.0,0.0,,False,Нет,Нет
1,Рунический доспех,"Базовое свойство: +10% к уклонению, если значение Воли больше 50.",,,Xeno,Pattern,Light,30.0,0.0,,False,Нет,Нет


### Prepare enemy ships data for saving

In [374]:
display(enemies_stats_eng.head(2), enemies_stats_ru.head(2))

,Ship_Name,Difficulty_Type,Weapons,Weapon_Slot,Weapon_Name,Weapon_Descr,Weapon_Type,Ability_Range,Ammo_Damage,Ammo_Max_Damage,Damage_Instances,Ammo_Damage_Type,Ship_Stats,Starship_Speed,Initiative,Evasion,Hull_Integrity,Armour_Aft,Armour_Fore,Armour_Port,Armour_Starboard,Hit_Chance,Crit_Chance,Shields_Stats,Enemy_Desired_DistanceTo_Player,Scrap_Reward,Size
0,Aeldari Aconite,Hard,Weapons:,Prow,Pulsar Lance,A long-ranged lance weapon dealing 84 damage to a chosen enemy within a narrow firing arc.,Lances,15,28.0,0.0,3,Energy,Other stats:,16.0,40.0,25.0,146.0,4.0,4.0,4.0,4.0,95.0,5.0,"Fore: 0, Port: 0, Starboard: 0, Aft: 0,",6.0,33.0,Frigate_1x2
1,,,,Prow,Aspect Torpedo Tubes,Torpedo tubes capable of firing 5 neutron torpedoes per salvo.\nReload: 1 round,TorpedoTubes,15,24.0,0.0,1,Energy,,,,,,,,,,,,,,,


,Название,Сложность,Вооружение,Позиция_Оружия,Название_Оружия,Описание_Оружия,Тип_Оружия,Дальность_Оружия,Урон_Снаряда,Макс_Урон_Снаряда,Количество_Залпов,Снаряд_Тип_Урона,Характеристики,Скорость,Инициатива,Уклонение,Целостность_Корпуса,Броня_Корма,Броня_Нос,Броня_ЛевыйБорт,Броня_ПравыйБорт,Шанс_Попадания,Шанс_КритПопадания,Щиты,Желаемая_ДистанцияДо_Игрока,Награда_Металлолом,Размер
0,"Эльдарский фрегат класса ""Аконит""",Hard,Вооружение:,Нос,Пульсарное копье,"Дальнобойный лэнс-излучатель, который наносит 64 урона выбранному врагу в узком секторе стрельбы.",Lances,15,28.0,0.0,3,Энергетический,Характеристики:,16.0,40.0,25.0,146.0,4.0,4.0,4.0,4.0,95.0,5.0,"Нос: 0, Левый борт: 0, Правый борт: 0, Корма: 0,",6.0,33.0,Frigate_1x2
1,,,,Нос,Аспектный торпедный аппарат,"Аппарат, выпускающий 5 нейтронных торпед за залп.\nПерезарядка: 1 раунд",TorpedoTubes,15,24.0,0.0,1,Энергетический,,,,,,,,,,,,,,,


### Format ```class_ability_types``` table.

In [375]:
display(class_ability_types.head())

,Class_Name,Class_Name_Ru,Ability_Type,Feature_Id,Ability_Name,Ability_Name_Ru,Feature_Class,Ability_Descr,Ability_Descr_Ru,Ability_Tag_0,Ability_Tag_1,Ability_Tag_2,Ability_Tag_3
0,Operative,Оперативник,ActiveAbility,d9e00160ce96479281c1abe9a26ac4e5,Precise Attack,Выверенная атака,Adept,"For the next attack against a target affected by an exploit, that target's cover efficiency will be reduced by −(5 + bonus Perception × stacks of exploit)%. This attack will have +(5 + bonus Perception × stacks of exploit)% hit chance.","При следующей атаке против цели, на которую наложена уязвимость, эффективность укрытия этой цели уменьшается на (5 + бонус Восприятие × число уровней эффекта ""уязвимость"")%. Шанс попадания этой атаки увеличивается на (5 + бонус Восприятие × число уровней эффекта ""уязвимость"")%.",Offense,Archetype,,
1,Operative,Оперативник,ActiveAbility,4d68f05c9aaf4620aa66f16cd4ba2efd,Perfect Spot,Идеальная позиция,Adept,"While in cover, the Operative gains +25% cover efficiency, +15 Perception, and +15 Ballistic Skill.\nThis effect lasts until the Operative moves. Does not stack.","Находясь в укрытии, оперативник получает +25% к эффективности укрытия, +15 к Восприятию и +15 к Дальнему Бою.\nДействует, пока оперативник не начнет движение. Эффект не суммируется.",Offense,Archetype,,
2,Operative,Оперативник,ActiveAbility,ec872eda7b1b40a08122b6330262a903,Intimidation,Устрашение,Adept,The Operative's next non-area attack inflicts the intimidated effect on the first target hit by the attack and all enemies in a 5-cell radius around that target.\nThe damage dealt by intimidated enemies is reduced by −(2 × PER bonus)% until the Operative's next turn. This negative effect is doub...,"Следующая атака оперативника (кроме атак по области) накладывает эффект ""испуг"" на первую цель, получившую попадание, и на всех противников в радиусе 5 клеток от нее.\nИспуганные противники наносят на (2 × бонус ВОС)% меньше урона до следующего хода оперативника. Этот негативный эффект удваивает...",Support,Archetype,,
3,Operative,Оперативник,ActiveAbility,65f2a606086249969be80163be79e32c,Joint Analysis,Поделиться знаниями,Adept,"Until the Operative's next turn, their allies' attacks also remove exploits and deal + bonus Intelligence more damage.",До следующего хода оперативника атаки его союзников также снимают уязвимости и наносят на бонус Интеллект урона больше.,Support,Archetype,,
4,Operative,Оперативник,ActiveAbility,9f2261fcdb7a494eb4a59966ac59c795,Tactical Knowledge,Понимание тактики,Adept,"The Operative removes −1 exploit from each enemy in a 5-cell radius around a point within 10 cells from the Operative. If there is only one enemy in the area, the Operative removes all exploit stacks from that enemy.\nAll allies gain +(2 × number of removed exploits)% armour until the end of com...","Оперативник снимает по 1 уязвимости с каждого противника в радиусе 5 клеток от точки, расположенной на расстоянии до 10 клеток от себя. Если в этой области находится только один противник, оперативник снимает с него все уязвимости.\nВсе союзники получают +(2 × число снятых уровней эффекта ""уязви...",Support,Archetype,,


Separate into En and Ru versions:

In [376]:
english_version_cols = [ 'Class_Name',	'Ability_Type',	
                       	 'Ability_Name', 'Ability_Descr',                            	
						'Ability_Tag_0',	'Ability_Tag_1',
                        'Ability_Tag_2',	'Ability_Tag_3',]
russian_version_cols = [ 'Class_Name_Ru',	'Ability_Type',	
                       	 'Ability_Name_Ru', 'Ability_Descr_Ru',                            	
						'Ability_Tag_0',	'Ability_Tag_1',
                        'Ability_Tag_2',	'Ability_Tag_3', ]
class_ability_types_en = class_ability_types[english_version_cols].copy(deep = True)
class_ability_types_ru = class_ability_types[russian_version_cols].copy(deep=True)


Apply format to russian headings:

In [377]:
ru_column_names = {'Class_Name_Ru':'Название_Класса',
                   'Ability_Type':'Тип_Способности',
                   'Ability_Name_Ru':'Название_Способности',
                   'Ability_Descr_Ru':'Описание_Способности',
                   'Ability_Tag_0':'Тег_Способности_0',
                   'Ability_Tag_1':'Тег_Способности_1',
                   'Ability_Tag_2':'Тег_Способности_2',
                   'Ability_Tag_3':'Тег_Способности_3',
                   'Is_ClassAbility':'Это_Способность_Класса',
}
class_ability_types_ru = class_ability_types_ru.rename(columns = ru_column_names)

In [378]:
class_ability_types_ru['Тип_Способности'].unique()

array(['ActiveAbility', 'Keystone Feature', 'Talent', 'AscensionTalent',
       'UltimateAbility', 'AP Increase', 'Characteristic Increase'],
      dtype=object)

Rename and replace values:

In [379]:
class_ability_types_en['Ability_Type'] = class_ability_types_en['Ability_Type'].replace({'AscensionTalent':'ExemplarTalent'})
class_ability_types_ru['Название_Класса'] = class_ability_types_ru['Название_Класса'].replace({'AP':'Очки движения'})
class_ability_types_ru['Тип_Способности'] = class_ability_types_ru['Тип_Способности'].replace(
    {'AscensionTalent':'ExemplarTalent',
     'ActiveAbility':'Активная способность',
     'Talent':'Талант',
     'AscensionTalent':'Талант гранд-магистра',
     'UltimateAbility':'Особая способность',
     'Keystone Feature':'Отличительная черта',
     'AP Increase':'Увеличение ОД',
     'Characteristic Increase':'Увеличение характеристик',
     })
class_ability_types_ru['Тег_Способности_0'] = class_ability_types_ru['Тег_Способности_0'].replace(
    {'Offense':'Атакующие способности',
     'Defense':'Защитные способности',
     'Support':'Способности поддержки',
     'Universal':'Универсальные способности',
     'Archetype':'Таланты архетипов',
     'Warp':'Варп-способности',
})
class_ability_types_ru['Тег_Способности_1'] = class_ability_types_ru['Тег_Способности_1'].replace(
    {'Offense':'Атакующие способности',
     'Defense':'Защитные способности',
     'Support':'Способности поддержки',
     'Universal':'Универсальные способности',
     'Archetype':'Таланты архетипов',
     'Warp':'Варп-способности',
})
class_ability_types_ru['Тег_Способности_2'] = class_ability_types_ru['Тег_Способности_2'].replace(
    {'Offense':'Атакующие способности',
     'Defense':'Защитные способности',
     'Support':'Способности поддержки',
     'Universal':'Универсальные способности',
     'Archetype':'Таланты архетипов',
     'Warp':'Варп-способности',
})
class_ability_types_ru['Тег_Способности_3'] = class_ability_types_ru['Тег_Способности_3'].replace(
    {'Offense':'Атакующие способности',
     'Defense':'Защитные способности',
     'Support':'Способности поддержки',
     'Universal':'Универсальные способности',
     'Archetype':'Таланты архетипов',
     'Warp':'Варп-способности',
})


Final look:

In [380]:
display(class_ability_types_ru.head(2),
        class_ability_types_en.head(2))

,Название_Класса,Тип_Способности,Название_Способности,Описание_Способности,Тег_Способности_0,Тег_Способности_1,Тег_Способности_2,Тег_Способности_3
0,Оперативник,Активная способность,Выверенная атака,"При следующей атаке против цели, на которую наложена уязвимость, эффективность укрытия этой цели уменьшается на (5 + бонус Восприятие × число уровней эффекта ""уязвимость"")%. Шанс попадания этой атаки увеличивается на (5 + бонус Восприятие × число уровней эффекта ""уязвимость"")%.",Атакующие способности,Таланты архетипов,,
1,Оперативник,Активная способность,Идеальная позиция,"Находясь в укрытии, оперативник получает +25% к эффективности укрытия, +15 к Восприятию и +15 к Дальнему Бою.\nДействует, пока оперативник не начнет движение. Эффект не суммируется.",Атакующие способности,Таланты архетипов,,


,Class_Name,Ability_Type,Ability_Name,Ability_Descr,Ability_Tag_0,Ability_Tag_1,Ability_Tag_2,Ability_Tag_3
0,Operative,ActiveAbility,Precise Attack,"For the next attack against a target affected by an exploit, that target's cover efficiency will be reduced by −(5 + bonus Perception × stacks of exploit)%. This attack will have +(5 + bonus Perception × stacks of exploit)% hit chance.",Offense,Archetype,,
1,Operative,ActiveAbility,Perfect Spot,"While in cover, the Operative gains +25% cover efficiency, +15 Perception, and +15 Ballistic Skill.\nThis effect lasts until the Operative moves. Does not stack.",Offense,Archetype,,


___________

In [381]:
class_ability_types_ru['Тег_Способности_0'].unique()

array(['Атакующие способности', 'Способности поддержки',
       'Защитные способности', '', 'Универсальные способности',
       'Таланты архетипов', 'Варп-способности'], dtype=object)

### Format homeworld talents table

In [382]:
display(homeworld_talents.head(1))

,AssetId,Feature_Name,Feature_Name_Ru,World,Feature_Descr,Feature_Descr_Ru,FeatureDescr_Form_En,FeatureDescr_Form_Ru,Is_Talent,Filepath
0,56513668e80d49c1ac337b170b863d45,Survival Instinct,Инстинкт выживания,Death,"Once per combat, when the {g|Encyclopedia:HitPoints}wounds{/g} of a death world character drop below 30%, they gain +20% of their {g|Encyclopedia:MaxWounds}maximum wounds{/g} as {g|Encyclopedia:TemporaryWounds}temporary wounds{/g}.\nDeath world characters also gain a +20% {g|Encyclopedia:Bonus}b...","Один раз за бой, когда {g|Encyclopedia:HitPoints}здоровье{/g} персонажа с мира смерти опускается ниже 30%, он получает +20% от {g|Encyclopedia:MaxWounds}максимального здоровья{/g} в качестве {g|Encyclopedia:TemporaryWounds}временного здоровья{/g}.\nПерсонаж с мира смерти также получает {g|Encycl...","Once per combat, when the wounds of a death world character drop below 30%, they gain +20% of their maximum wounds as temporary wounds.\nDeath world characters also gain a +20% bonus to dodge and armour while they have at least 1 temporary wound from any source.","Один раз за бой, когда здоровье персонажа с мира смерти опускается ниже 30%, он получает +20% от максимального здоровья в качестве временного здоровья.\nПерсонаж с мира смерти также получает бонус +20% к уклонению и броне, когда у него есть хотя бы 1 временное здоровье, полученное любым способом.",True,e:\Steam\RogueTrader\WorkFolder\Blueprints\Backgrounds\Homeworlds\DeathWorld\DeathWorld_InnateTalent.jbp


English version:

In [383]:
eng_columns = ['Feature_Name','World','FeatureDescr_Form_En']
homeworld_talents_eng = homeworld_talents[eng_columns].copy(deep = True)
#Rename columns
homeworld_talents_eng = homeworld_talents_eng.rename(columns = {'Feature_Name':'Talent',
                                                                'FeatureDescr_Form_En':'Description',
                                                                })


Russian version:

In [384]:
rus_columns = ['Feature_Name_Ru','World','FeatureDescr_Form_Ru']
homeworld_talents_ru = homeworld_talents[rus_columns]
#Rename columns
homeworld_talents_ru = homeworld_talents_ru.rename(columns = {'Feature_Name_Ru':'Талант',
                                                                'FeatureDescr_Form_Ru':'Описание',
                                                                'World':'Мир'
                                                                })
# Rename world names
ru_worlds = {'Death':'Мир смерти',
             'Forge':'Мир-кузня',
             'Fortress':'Мир-крепость',
             'Hive':'Мир-улей',
             'Imperial':'Имперский мир',
             'Void':'Пустотнорожденный',
}
homeworld_talents_ru['Мир'] = homeworld_talents_ru['Мир'].map(ru_worlds)

Final look:

In [385]:
display('English version'.center(30),
    homeworld_talents_eng.head(),
        'Russian'.center(30),
        homeworld_talents_ru.head())

'       English version        '

,Talent,World,Description
0,Survival Instinct,Death,"Once per combat, when the wounds of a death world character drop below 30%, they gain +20% of their maximum wounds as temporary wounds.\nDeath world characters also gain a +20% bonus to dodge and armour while they have at least 1 temporary wound from any source."
1,Brutal Hunter,Death,Death world characters gain +15% critical hit chance against targets with the bleeding effect or with 50% or less of their maximum wounds.
2,Trusty Weapons,Death,"Death world characters gain +10% to critical hit chance for axes, hammers, and las weapons."
3,Wounded Beast,Death,Every injury increases the death world character's Agility and Willpower by +5.\nEvery trauma counts as 3 injuries.
4,Lifetime of Scars,Death,"Death world characters suffer − less damage from fire, toxic damage, and bleeding effects."


'           Russian            '

,Талант,Мир,Описание
0,Инстинкт выживания,Мир смерти,"Один раз за бой, когда здоровье персонажа с мира смерти опускается ниже 30%, он получает +20% от максимального здоровья в качестве временного здоровья.\nПерсонаж с мира смерти также получает бонус +20% к уклонению и броне, когда у него есть хотя бы 1 временное здоровье, полученное любым способом."
1,Безжалостный охотник,Мир смерти,"Персонаж с мира смерти получает +15% к шансу критического попадания против целей, страдающих от эффекта кровотечения, и целей, у которых меньше 50% от максимального запаса здоровья."
2,Проверенное оружие,Мир смерти,"Персонаж с мира смерти получает +10% к шансу критического попадания, когда использует топоры, молоты или лазерное оружие."
3,Раненый зверь,Мир смерти,Каждое ранение увеличивает Ловкость и Волю персонажа с мира смерти на 5.\nКаждая травма считается за 3 ранения.
4,Опыт выживания,Мир смерти,"Персонаж родом с мира смерти получает на меньше огненного и токсичного урона, а также урона от кровотечения."


In [386]:
homeworld_talents_ru['Мир'].unique()

array(['Мир смерти', 'Мир-кузня', 'Мир-крепость', 'Мир-улей',
       'Имперский мир', 'Пустотнорожденный'], dtype=object)

## Appendix

<span style="color:yellow"> Find file by AssetId:</span>

In [387]:
assets_df.query('AssetId=="de5bd3fa3ba642208ede17ed3827e29e"')['File']

3768    e:\Steam\RogueTrader\WorkFolder\Blueprints\Consumables\Drugs\UncannySkin.jbp
Name: File, dtype: object

In [388]:
index_df.query('index == "dd529468-40fb-48e8-9a42-6f18c9f5910e"')['text']

44671    Grants +30% {g|Encyclopedia:WarhammerArmourGlossary}armour{/g} and inflicts a −20 {g|Encyclopedia:Penalty}penalty{/g} to {g|Encyclopedia:WarhammerAgility}Agility{/g} for 2 {g|Encyclopedia:Round}rounds{/g}.
Name: text, dtype: object

### Saving tables

#### English version

In [389]:
# Save folder of ready tables
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
# excel writer
writer = pd.ExcelWriter(save_folder + r'\rogue_trader_items_eng.xlsx')
# excel workbook
workbook = writer.book

Style format options:

In [390]:
#headers style
header_format = workbook.add_format({
        'bold':True,
        'text_wrap':True,
        'valign':'top',
        'align':'center',
        'fg_color':'#d5c4a5'
    })
# cells style
cell_names = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter',
                                       'bold':True})

cell_format = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter'})
cell_format2 = workbook.add_format({'align':'center',
                                       'valign':'vcenter'})
# Apply conditional formatting
cell_red = workbook.add_format({'bg_color':'#9C0006',
                                'align':'center',
                                 'valign':'vcenter' })
cell_light_red = workbook.add_format({'bg_color':'#e06666',
                                'align':'center',
                                 'valign':'vcenter' })
cell_yellow = workbook.add_format({'bg_color':'#eece6c',
                                'align':'center',
                                 'valign':'vcenter' })
cell_gray = workbook.add_format({'bg_color':'#999999',
                                'align':'center',
                                 'valign':'vcenter' })
#
cell_green = workbook.add_format({'bg_color':'#b6d7a8',
                                'align':'center',
                                 'valign':'vcenter' })
cell_blue = workbook.add_format({'bg_color':'#6fa8dc',
                                'align':'center',
                                 'valign':'vcenter' })

##### Weapons table

In [391]:
# WEAPONS
weapons_table.to_excel(writer, sheet_name='Weapons', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Weapons']

#Format header
for col_num, value in enumerate(weapons_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',400/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:E',100/x, cell_format2)
worksheet.set_column('F:F',80/x, cell_format2)
worksheet.set_column('G:H',100/x, cell_format2)
worksheet.set_column('I:I',80/x, cell_format2)
worksheet.set_column('J:J',100/x, cell_format2)
worksheet.set_column('K:T',80/x, cell_format2)
    ## ability 1 group
worksheet.set_column('U:U',85/x, cell_format)
worksheet.set_column('V:V',200/x, cell_format)
worksheet.set_column('W:W',80/x, cell_format2)
worksheet.set_column('X:X',92/x, cell_format2)
    ## ability 2 group
worksheet.set_column('Y:Y',85/x, cell_format)
worksheet.set_column('Z:Z',200/x, cell_format)
worksheet.set_column('AA:AA',80/x, cell_format2)
worksheet.set_column('AB:AB',92/x, cell_format2)
    ## ability 3 group
worksheet.set_column('AC:AC',85/x, cell_format)
worksheet.set_column('AD:AD',200/x, cell_format)
worksheet.set_column('AE:AE',80/x, cell_format2)
worksheet.set_column('AF:AF',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AG:AG',85/x, cell_format)
worksheet.set_column('AH:AH',200/x, cell_format)
worksheet.set_column('AI:AI',80/x, cell_format2)
worksheet.set_column('AJ:AJ',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AK:AK',85/x, cell_format)
worksheet.set_column('AL:AL',200/x, cell_format)
worksheet.set_column('AM:AM',80/x, cell_format2)
worksheet.set_column('AN:AN',92/x, cell_format2)
# suppot columns
worksheet.set_column('AO:AQ',100/x, cell_format2)
worksheet.set_column('AR:AR',200/x, cell_format)

#Apply conditional formatting
worksheet.conditional_format('C2:D420',{'type':'no_blanks',
                                    'format':cell_green})
    

0

##### Armors

In [392]:
armors_table.to_excel(writer, sheet_name='Armors', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Armors']

#Format header
for col_num, value in enumerate(armors_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',400/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)

## Armor stats columns
worksheet.set_column('E:J',90/x, cell_format2)
#boolean columns
worksheet.set_column('K:M',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D110',{'type':'no_blanks',
                                    'format':cell_green})

0

##### Other equipment, english

In [393]:
dfs = [shoulders_table, feet_table, gloves_table, headgear_table, necklaces_table, rings_table]

##SHOULDERS
shoulders_table.to_excel(writer, sheet_name='Shoulders', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Shoulders']
#Format header
for col_num, value in enumerate(shoulders_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:G',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## FEET
feet_table.to_excel(writer, sheet_name='Feet', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Feet']
#Format header
for col_num, value in enumerate(feet_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## GLOVES
gloves_table.to_excel(writer, sheet_name='Gloves', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Gloves']
#Format header
for col_num, value in enumerate(gloves_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## HEADGEAR
headgear_table.to_excel(writer, sheet_name='Headgear', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Headgear']
#Format header
for col_num, value in enumerate(headgear_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D70',{'type':'no_blanks',
                                    'format':cell_green})

## NECKLACES
necklaces_table.to_excel(writer, sheet_name='Necklaces', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Necklaces']
#Format header
for col_num, value in enumerate(necklaces_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
worksheet.set_column('I:I',130/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})
## RINGS
rings_table.to_excel(writer, sheet_name='Rings', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Rings']
#Format header
for col_num, value in enumerate(rings_table.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',130/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:I',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D120',{'type':'no_blanks',
                                    'format':cell_green})


0

##### Ships

In [394]:
#enemies_stats_eng.to_excel(writer, sheet_name='EnemyShips', engine ='xlsxwriter', freeze_panes=(1,1) )
## SHIPS table redo when adjust table 
enemies_stats_eng.to_excel(writer, sheet_name='EnemyShips', engine ='xlsxwriter', startrow = 1, 
                           header = False, freeze_panes=(1,1), index = None)
worksheet = writer.sheets['EnemyShips']

#Format header
for col_num, value in enumerate(enemies_stats_eng.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',70/x, cell_format2)
worksheet.set_column('C:C',80/x, cell_names)
worksheet.set_column('D:D',65/x, cell_format2)
worksheet.set_column('E:E',130/x, cell_format)
worksheet.set_column('F:F',450/x, cell_format)
worksheet.set_column('G:G',120/x, cell_format)
worksheet.set_column('H:L',80/x, cell_format2)
worksheet.set_column('M:M',80/x, cell_names)
worksheet.set_column('N:U',65/x, cell_format2)
worksheet.set_column('V:W',80/x, cell_format2)
worksheet.set_column('X:X',130/x, cell_format)
worksheet.set_column('Y:AA',100/x, cell_format)
# Conditional formatting
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'MiniBoss',
                                   'format':cell_red})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Elite',
                                   'format':cell_light_red})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Hard',
                                   'format':cell_yellow})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Common',
                                   'format':cell_gray})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Swarm',
                                   'format':cell_blue})
worksheet.conditional_format('H2:H100',{'type':'no_blanks',
                                    'format':cell_green})
worksheet.conditional_format('N2:N100',{'type':'no_blanks',
                                    'format':cell_green})
# apply format to damage type columns
cell_energy = workbook.add_format({'bg_color':'#ffe599',
                                'align':'center',
                                 'valign':'vcenter' }) 
cell_kinetic = workbook.add_format({'bg_color':'#5b5b5b',
                                'align':'center',
                                 'valign':'vcenter' }) 
worksheet.conditional_format('L2:L100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Energy',
                                   'format':cell_energy})
worksheet.conditional_format('L2:L100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Kinetic',
                                   'format':cell_energy})                                     

0

##### Class abilities and homeworld talents, english:

In [395]:
class_ability_types_en.to_excel(writer, sheet_name='Class abilities', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Class abilities']
#Format header
for col_num, value in enumerate(class_ability_types_en.columns.values):
        worksheet.write(0,col_num, value, header_format)
# adjust column width
x = 7.2 # width coeff
worksheet.set_column('A:C',120/x, cell_format)
worksheet.set_column('D:D',400/x, cell_format)
worksheet.set_column('E:I',120/x, cell_format)

homeworld_talents_eng.to_excel(writer, sheet_name='Homeworld talents', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Homeworld talents']
#Format header
for col_num, value in enumerate(homeworld_talents_eng.columns.values):
        worksheet.write(0,col_num, value, header_format)
# adjust column width
x = 7.2 # width coeff
worksheet.set_column('A:B',120/x, cell_format)
worksheet.set_column('C:C',450/x, cell_format)



0

Close saving:

In [396]:
writer.save()

#### Russian version

In [397]:
# Save folder of ready tables
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
# excel writer
writer = pd.ExcelWriter(save_folder + r'\rogue_trader_items_ru.xlsx')
# excel workbook
workbook = writer.book

In [398]:
#headers style
header_format = workbook.add_format({
        'bold':True,
        'text_wrap':True,
        'valign':'top',
        'align':'center',
        'fg_color':'#d5c4a5'
    })
# cells style
cell_format = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter'})
cell_names = workbook.add_format({'text_wrap':True,
                                       'align':'center',
                                       'valign':'vcenter',
                                       'bold':True})
cell_format2 = workbook.add_format({'align':'center',
                                       'valign':'vcenter'})
# Apply conditional formatting
cell_red = workbook.add_format({'bg_color':'#9C0006',
                                'align':'center',
                                 'valign':'vcenter' })
cell_light_red = workbook.add_format({'bg_color':'#e06666',
                                'align':'center',
                                 'valign':'vcenter' })
cell_yellow = workbook.add_format({'bg_color':'#eece6c',
                                'align':'center',
                                 'valign':'vcenter' })
cell_gray = workbook.add_format({'bg_color':'#999999',
                                'align':'center',
                                 'valign':'vcenter' })
#
cell_green = workbook.add_format({'bg_color':'#b6d7a8',
                                'align':'center',
                                 'valign':'vcenter' })
cell_blue = workbook.add_format({'bg_color':'#6fa8dc',
                                'align':'center',
                                 'valign':'vcenter' })

In [399]:
##### Weapons

In [400]:
# WEAPONS
weapons_table_ru.to_excel(writer, sheet_name='Оружие', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Оружие']

#Format header
for col_num, value in enumerate(weapons_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',400/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:E',100/x, cell_format2)
worksheet.set_column('F:F',80/x, cell_format2)
worksheet.set_column('G:H',100/x, cell_format2)
worksheet.set_column('I:I',80/x, cell_format2)
worksheet.set_column('J:J',100/x, cell_format2)
worksheet.set_column('K:T',80/x, cell_format2)
    ## ability 1 group
worksheet.set_column('U:U',85/x, cell_format)
worksheet.set_column('V:V',200/x, cell_format)
worksheet.set_column('W:W',80/x, cell_format2)
worksheet.set_column('X:X',92/x, cell_format2)
    ## ability 2 group
worksheet.set_column('Y:Y',85/x, cell_format)
worksheet.set_column('Z:Z',200/x, cell_format)
worksheet.set_column('AA:AA',80/x, cell_format2)
worksheet.set_column('AB:AB',92/x, cell_format2)
    ## ability 3 group
worksheet.set_column('AC:AC',85/x, cell_format)
worksheet.set_column('AD:AD',200/x, cell_format)
worksheet.set_column('AE:AE',80/x, cell_format2)
worksheet.set_column('AF:AF',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AG:AG',85/x, cell_format)
worksheet.set_column('AH:AH',200/x, cell_format)
worksheet.set_column('AI:AI',80/x, cell_format2)
worksheet.set_column('AJ:AJ',92/x, cell_format2)
    ## ability 4 group
worksheet.set_column('AK:AK',85/x, cell_format)
worksheet.set_column('AL:AL',200/x, cell_format)
worksheet.set_column('AM:AM',80/x, cell_format2)
worksheet.set_column('AN:AN',92/x, cell_format2)
# suppot columns
worksheet.set_column('AO:AQ',110/x, cell_format2)
worksheet.set_column('AR:AR',200/x, cell_format)

#Apply conditional formatting
worksheet.conditional_format('C2:D420',{'type':'no_blanks',
                                    'format':cell_green})
    

0

##### Armors

In [401]:
armors_table_ru.to_excel(writer, sheet_name='Броня', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Броня']

#Format header
for col_num, value in enumerate(armors_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
#set column width
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',400/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
## Armor stats columns
worksheet.set_column('E:J',90/x, cell_format2)
#boolean columns
worksheet.set_column('K:M',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D110',{'type':'no_blanks',
                                    'format':cell_green})

0

##### Other equipment, english

In [402]:
dfs = [shoulders_table, feet_table, gloves_table, headgear_table, necklaces_table, rings_table]

##SHOULDERS
shoulders_table_ru.to_excel(writer, sheet_name='Накидки', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Накидки']
#Format header
for col_num, value in enumerate(shoulders_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:G',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## FEET
feet_table_ru.to_excel(writer, sheet_name='Сапоги', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Сапоги']
#Format header
for col_num, value in enumerate(feet_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## GLOVES
gloves_table_ru.to_excel(writer, sheet_name='Перчатки', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Перчатки']
#Format header
for col_num, value in enumerate(gloves_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})

## HEADGEAR
headgear_table_ru.to_excel(writer, sheet_name='Головные уборы', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Головные уборы']
#Format header
for col_num, value in enumerate(headgear_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D70',{'type':'no_blanks',
                                    'format':cell_green})

## NECKLACES
necklaces_table_ru.to_excel(writer, sheet_name='Ожерелья', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Ожерелья']
#Format header
for col_num, value in enumerate(necklaces_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:H',90/x, cell_format2)
worksheet.set_column('I:I',130/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D60',{'type':'no_blanks',
                                    'format':cell_green})
## RINGS
rings_table_ru.to_excel(writer, sheet_name='Кольца', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Кольца']
#Format header
for col_num, value in enumerate(rings_table_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',450/x, cell_format)
worksheet.set_column('C:C',137/x, cell_format)
worksheet.set_column('D:D',170/x, cell_format)
worksheet.set_column('E:I',90/x, cell_format2)
#Apply conditional formatting
worksheet.conditional_format('C2:D120',{'type':'no_blanks',
                                    'format':cell_green})


0

##### Ships

In [403]:
#enemies_stats_ru.to_excel(writer, sheet_name='Корабли', engine ='xlsxwriter',freeze_panes=(1,1) )
enemies_stats_ru.to_excel(writer, sheet_name='Корабли', engine ='xlsxwriter', startrow = 1, 
                           header = False, freeze_panes=(1,1), index = None )
worksheet = writer.sheets['Корабли']

#Format header
for col_num, value in enumerate(enemies_stats_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_names)
worksheet.set_column('B:B',75/x, cell_format2)
worksheet.set_column('C:C',90/x, cell_names)
worksheet.set_column('D:D',75/x, cell_format)
worksheet.set_column('E:E',130/x, cell_format)
worksheet.set_column('F:F',450/x, cell_format)
worksheet.set_column('G:G',120/x, cell_format)
worksheet.set_column('H:K',80/x, cell_format2)
worksheet.set_column('L:L',100/x, cell_format2)
worksheet.set_column('M:M',120/x, cell_names)
worksheet.set_column('N:U',96/x, cell_format2)
worksheet.set_column('V:W',80/x, cell_format2)
worksheet.set_column('X:X',130/x, cell_format)
worksheet.set_column('Y:AA',100/x, cell_format)
# Conditional formatting
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'MiniBoss',
                                   'format':cell_red})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Elite',
                                   'format':cell_light_red})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Hard',
                                   'format':cell_yellow})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Common',
                                   'format':cell_gray})
worksheet.conditional_format('B2:B100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Swarm',
                                   'format':cell_blue})
worksheet.conditional_format('H2:H100',{'type':'no_blanks',
                                    'format':cell_green})
worksheet.conditional_format('N2:N100',{'type':'no_blanks',
                                    'format':cell_green})
# apply format to damage type columns
cell_energy = workbook.add_format({'bg_color':'#ffe599',
                                'align':'center',
                                 'valign':'vcenter' }) 
cell_kinetic = workbook.add_format({'bg_color':'#5b5b5b',
                                'align':'center',
                                 'valign':'vcenter' }) 
worksheet.conditional_format('L2:L100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Энергетический',
                                   'format':cell_energy})
worksheet.conditional_format('L2:L100', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Кинетический',
                                   'format':cell_energy}) 


0

In [404]:
enemies_stats_ru['Сложность'].unique()

array(['Hard', '', 'Common', 'Elite', 'MiniBoss', 'Swarm'], dtype=object)

##### Class abilities and homeworld talents, english:

In [405]:
class_ability_types_ru.to_excel(writer, sheet_name='Классовые умения', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Классовые умения']
#Format header
for col_num, value in enumerate(class_ability_types_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# adjust column width
x = 7.2 # width coeff
worksheet.set_column('A:C',120/x, cell_format)
worksheet.set_column('D:D',400/x, cell_format)
worksheet.set_column('E:I',120/x, cell_format)

homeworld_talents_ru.to_excel(writer, sheet_name='Таланты происхождения', engine ='xlsxwriter', startrow = 1, 
                           header = False, index = None, freeze_panes=(1,1) )
worksheet = writer.sheets['Таланты происхождения']
#Format header
for col_num, value in enumerate(homeworld_talents_ru.columns.values):
        worksheet.write(0,col_num, value, header_format)
# adjust column width
x = 7.2 # width coeff
worksheet.set_column('A:B',120/x, cell_format)
worksheet.set_column('C:C',450/x, cell_format)



0

Close saving:

In [406]:
writer.save()

#english version 
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
dfs = [armors_table, shoulders_table, feet_table, gloves_table, headgear_table, necklaces_table, rings_table]
with pd.ExcelWriter(save_folder + r'\rogue_trader_items.xlsx') as writer:
#weapons
    weapons_table.to_excel(writer, sheet_name='Weapons', index=False)  
#armors
    armors_table.to_excel(writer, sheet_name='Armors', index=False)
#shoulders
    shoulders_table.to_excel(writer, sheet_name='Shoulders', index=False)
#Feet
    feet_table.to_excel(writer, sheet_name='Feet', index=False)
#Gloves
    gloves_table.to_excel(writer, sheet_name='Gloves', index=False)
#Headgear
    headgear_table.to_excel(writer, sheet_name='Headgear', index=False)
#Necklaces
    necklaces_table.to_excel(writer, sheet_name='Necklaces', index=False)
#Rings
    rings_table.to_excel(writer, sheet_name='Rings', index=False)
#Ships
    enemies_stats_eng.to_excel(writer, sheet_name='EnemyShips')
#Player abilities
    class_ability_types_en.to_excel(writer, sheet_name='ClassAbilities')
    


#Russian version
save_folder = r'e:\Steam\RogueTrader\SaveFolder\ReadyTables'
dfs = [armors_table, shoulders_table, feet_table, gloves_table, headgear_table, necklaces_table, rings_table]
with pd.ExcelWriter(save_folder + r'\rogue_trader_items_ru.xlsx') as writer:
#weapons
    weapons_table_ru.to_excel(writer, sheet_name='Оружие', index=False)  
#armors
    armors_table_ru.to_excel(writer, sheet_name='Броня', index=False)
#shoulders
    shoulders_table_ru.to_excel(writer, sheet_name='Накидки', index=False)
#Feet
    feet_table_ru.to_excel(writer, sheet_name='Сапоги', index=False)
#Gloves
    gloves_table_ru.to_excel(writer, sheet_name='Перчатки', index=False)
#Headgear
    headgear_table_ru.to_excel(writer, sheet_name='Головные уборы', index=False)
#Necklaces
    necklaces_table_ru.to_excel(writer, sheet_name='Ожерелья', index=False)
#Rings
    rings_table_ru.to_excel(writer, sheet_name='Кольца', index=False)
#Ships
    enemies_stats_ru.to_excel(writer, sheet_name='Корабли') 
#Player abilities
    class_ability_types_ru.to_excel(writer, sheet_name='Классовые умения')

________________

### Lab

Find item location area

display(equip_items_df_ru.query('Item_Name!=""')[['Item_Name_Ru','Item_Name']].head())
print('Item_Name: ',equip_items_df_ru[equip_items_df_ru['Item_Name']==""].shape[0])
print('Item_Name_Ru: ', equip_items_df_ru[equip_items_df_ru['Item_Name_Ru']==""].shape[0])

## SHIPS table redo when adjust table 
enemies_stats_eng.to_excel(writer, sheet_name='EnemyShips', engine ='xlsxwriter', startrow = 1, 
                           header = False, freeze_panes=(1,1) )
worksheet = writer.sheets['EnemyShips']

#Format header
for col_num, value in enumerate(enemies_stats_eng.columns.values):
        worksheet.write(0,col_num, value, header_format)
# format columns
x = 7.2 # width coeff
worksheet.set_column('A:A',200/x, cell_format)
worksheet.set_column('B:B',90/x, cell_format2)
worksheet.set_column('C:C',110/x, cell_format2)
worksheet.set_column('A:A',100/x, cell_format2)
worksheet.set_column('E:M',90/x, cell_format2)
worksheet.set_column('N:N',130/x, cell_format)
worksheet.set_column('O:O',120/x, cell_format2)
worksheet.set_column('P:P',90/x, cell_format2)
worksheet.set_column('Q:R',100/x, cell_format2)
worksheet.set_column('S:S',90/x, cell_format2)
worksheet.set_column('T:T',200/x, cell_format)
worksheet.set_column('U:U',450/x, cell_format)
worksheet.set_column('V:AA',120/x, cell_format2)
# Conditional formatting
worksheet.conditional_format('B2:B91', {'type':'text',
                                   'criteria':'containing',
                                   'value':'MiniBoss',
                                   'format':cell_red})
worksheet.conditional_format('B2:B91', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Elite',
                                   'format':cell_light_red})
worksheet.conditional_format('B2:B91', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Hard',
                                   'format':cell_yellow})
worksheet.conditional_format('B2:B91', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Common',
                                   'format':cell_gray})
worksheet.conditional_format('B2:B91', {'type':'text',
                                   'criteria':'containing',
                                   'value':'Swarm',
                                   'format':cell_blue})
worksheet.conditional_format('D2:D92',{'type':'no_blanks',
                                    'format':cell_green})